In [29]:
!pip install langchain pypdf pymongo PyPDF2 boto3 botocore dotenv tabula-py


  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [75 lines of output]
  C:\Users\karol\Desktop\python-projects\pytorch-start\env\Lib\site-packages\setuptools\installer.py:27: SetuptoolsDeprecationWarning: setuptools.installer is deprecated. Requirements should be satisfied by a PEP 517 installer.
    warnings.warn(
    error: subprocess-exited-with-error
  
    python setup.py egg_info did not run successfully.
    exit code: 1
  
    [17 lines of output]
    Traceback (most recent call last):
      File "<string>", line 2, in <module>
      File "<pip-setuptools-caller>", line 14, in <module>
      File "C:\Users\karol\AppData\Local\Temp\pip-wheel-5ernexcl\distribute_1bcbf0477854411281825da787fdf69e\setuptools\__init__.py", line 2, in <module>
        from setuptools.extension import Extension, Library
      File "C:\Users\karol\AppData\Local\Temp\pip-wheel-5ernexcl\distribute_1bcbf0477854411281825da787fdf69e\setuptools\

In [1]:
!pip list

Package                                  Version            Editable project location
---------------------------------------- ------------------ -------------------------------------------------------------------------------
absl-py                                  1.3.0
aiobotocore                              2.15.1
aiofiles                                 23.2.1
aiohappyeyeballs                         2.4.0
aiohttp                                  3.10.5
aiohttp-cors                             0.7.0
aioitertools                             0.12.0
aiosignal                                1.2.0
aiosqlite                                0.20.0
alembic                                  1.13.2
aniso8601                                9.0.1
annotated-types                          0.7.0
anyio                                    4.6.0
apache-airflow                           2.10.1
apache-airflow-providers-common-compat   1.2.0
apache-airflow-providers-common-io       1.4.0
apache-airflow-

In [16]:
!pip install tabula-py pdfplumber langchain langchain-core langchain-openai langchain_community pandas unidecode

     ---------------------------------------- 0.0/235.5 kB ? eta -:--:--
     ---- -------------------------------- 30.7/235.5 kB 660.6 kB/s eta 0:00:01
     ------ ------------------------------ 41.0/235.5 kB 393.8 kB/s eta 0:00:01
     ------------------ ------------------- 112.6/235.5 kB 1.1 MB/s eta 0:00:01
     -------------------------------------- 235.5/235.5 kB 1.3 MB/s eta 0:00:00


In [20]:

import sys
import PyPDF2
import re
import pdfplumber
from langchain_core.tools import StructuredTool, tool
import os
from SecretManager import get_secret
from langchain_openai import ChatOpenAI
from enum import Enum
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Union
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor
from typing import Tuple
from langchain_core.language_models import BaseChatModel
from langchain_core.runnables.utils import Input
from abc import ABC


In [2]:
from unidecode import unidecode

In [4]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("D20200321Lj.pdf")
pages = loader.load()

In [16]:
!pwd

1055.24s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


/home/sagemaker-user/project-main


In [3]:
OPENAI_KEY = get_secret("OpenAiApiKey", os.getenv('REGION_NAME'))

print(OPENAI_KEY)

{'OPEN_API_KEY': 'sk-proj-43WipF94bBmSt0O2ZdWmT3BlbkFJBe8PXpIiiRW1vkmHckZI'}


In [4]:
openAIModel = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=OPENAI_KEY["OPEN_API_KEY"]
)

openAIModel4o = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    api_key=OPENAI_KEY["OPEN_API_KEY"]
)



In [5]:
class MethodEnum(str, Enum):
    add = "add"
    remove = "remove"
    change = "change"


class Pkt(BaseModel):
    pkt: list[str]


class Ust(BaseModel):
    ust: str
    pkt: list[str]


class Art(BaseModel):
    art: str
    usts: list[Ust]


In [26]:
# czy prawnicy głównie odwołowują się do artykółów z dzieników ustaw czy z monitorów polskich jakis innych dokumentów?
from langchain_core.prompts import ChatPromptTemplate

In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.document_transformers.openai_functions import (
    create_metadata_tagger
)

document_transformer = create_metadata_tagger(metadata_schema=table_extraction_schema, llm=openAIModel,
                                              prompt=ChatPromptTemplate.from_template("""
 You will get document page with text and/or with table, you need return normal text and you should summary the tables including all information provided in table, using full sentences and paste this summarization where the table was, return result without any explanation!!! give the plain text without any formation such as indices or new lines, don't omit section sign such as Art. , x),  x. or a) etc. in plain_text give the result with summarized tables using full sentences and place it correct place.

Only extract the properties mentioned in the 'information_extraction' function.

Passage:
{input}
"""))

NameError: name 'table_extraction_schema' is not defined

In [8]:
class PassageSplits(BaseModel):
    """Schema for splitting a legal act into passage, not subspoint, not subsection"""
    passage_splits: List[str]

In [8]:


passage_split_examples = [
    {
        "input": "Ustawa określa: 1) zasady, warunki i tryb podejmowania decyzji administracyjnej o objęciu refundacją leku, środka spożywczego specjalnego przeznaczenia żywi eniowego, wyrobu medycznego i uchylenia tej decyzji; 1a) zakres tajemnicy refundacyjnej; 2) zasady finansowania leku, środka spożywczego specjalnego przeznaczenia żywieniowego, wyrobu medycznego objętych decyzją, o której mowa w pkt 1; 3) kryteria tworzenia poziomów odpłatności i grup limitowych leków, środków spożywczych specjalnego przeznaczenia żywieniowego, wyrobów medycznych objętych decyzją, o której mowa w pkt 1;",
        "output": PassageSplits(passage_splits=[
            "Ustawa określa: 1) zasady, warunki i tryb podejmowania decyzji administracyjnej o objęciu refundacją leku, środka spożywczego specjalnego przeznaczenia żywi eniowego, wyrobu medycznego i uchylenia tej decyzji; 1a) zakres tajemnicy refundacyjnej; 2) zasady finansowania leku, środka spożywczego specjalnego przeznaczenia żywieniowego, wyrobu medycznego objętych decyzją, o której mowa w pkt 1; 3) kryteria tworzenia poziomów odpłatności i grup limitowych leków, środków spożywczych specjalnego przeznaczenia żywieniowego, wyrobów medycznych objętych decyzją, o której mowa w pkt 1;"])
    },
    {
        "input": "1. Ustala się urzędową marżę hurtową w wysokości 6% ceny zbytu netto leku, środka spożywczego specjalnego przeznaczenia żywieniowego lub wyrobu medycznego, nie niższą niż 50 groszy i nie wyższą niż 150 zł w przypadku, o którym mowa w art. 6 ust. 1 pkt 1 oraz 2000 zł w przypadkach, o których mowa w art. 6 ust. 1 pkt 2 i 3. 2. Przedsiębiorcy prowadzący obrót hurtowy w rozumieniu art. 72 ust. 3 ustawy z dnia 6 września 2001 r. – Prawo farmaceutyczne są obowiązani stosować cenę zbytu netto wraz z marżą, o której mowa w ust. 1, oraz właściwą stawkę podatku od towarów i usług niezależnie od miejsca dostawy leku, środka spożywczego specjalnego przeznaczenia żywieniowego lub wyrobu medycznego będącego przedmiotem tego obrotu. 3. Marża, o której mowa w ust. 1, może być dzielona pomiędzy przedsiębiorców prowadzących obrót hurtowy. Ustala się urzędową marżę detaliczną naliczaną od ceny hurtowej leku, środka spożywczego specjalnego przeznaczenia żywieniowego albo wyrobu medycznego stanowiącego podstawę limitu w danej grupie limitowej, w wysokości: od 5,00 zł do 40% marży, od 5,01 zł do 10,00 zł marża wynosi 2 zł + 30% ×(x −5,00 zł), od 10,01 zł do 20,00 zł marża wynosi 3,50 zł + 20% ×(x −10,00 zł), od 20,01 zł do 40,00 zł marża wynosi 5,50 zł + 15% ×(x −20,00 zł), od 40,01 zł do 80,00 zł marża wynosi 8,50 zł + 10% ×(x −40,00 zł), od 80,01 zł do 160,00 zł marża wynosi 12,50 zł + 5% ×(x −80,00 zł), od 160,01 zł do 320,00 zł marża wynosi 16,50 zł + 2,5% ×(x −160,00 zł), od 320,01 zł do 640,00 zł marża wynosi 20,50 zł + 2,5% ×(x −320,00 zł), od 640,01 zł do 1280,00 zł marża wynosi 28,50 zł + 2,5% ×(x −640,00 zł), a powyżej 1280,01 zł marża wynosi 44,50 zł + 1,25% ×(x −1280,00 zł), gdzie x oznacza cenę hurtową leku, środka spożywczego specjalnego przeznaczenia żywieniowego albo wyrobu medycznego stanowiącego podstawę limitu, uwzględniającą liczbę DDD leku, jednostek środka spożywczego specjalnego. przeznaczenia żywieniowego w opakow aniu albo liczbę jednostkowych wyrobów medycznych, albo liczbę jednostek wyrobu medycznego. Ustala się urzędową marżę detaliczną naliczaną od ceny hurtowej brutto leku, środka spożywczego specjalnego przeznaczenia żywieniowego albo wyrobu medycznego stanowiącego podstawę limitu w danej grupie limitowej, w wysokości: od 10,00 zł do 0,45 zł marża wynosi 35% * x, od 10,01 zł do 20,00 zł marża wynosi 3,95 zł + 25% * (x – 10,00 zł), od 20,01 zł do 40,00 zł marża wynosi 6,45 zł + 15% * (x – 20,00 zł), od 40,01 zł do 80,00 zł marża wynosi 9,45 zł + 10% * (x – 40,00 zł), od 80,01 zł do 160,00 zł marża wynosi 13,45 zł + 5% * (x – 80,00 zł), od 160,01 zł do 640,00 zł marża wynosi 17,45 zł + 2,75% * (x – 160,00 zł), a powyżej 640,01 zł marża wynosi 30,65 zł + 2% * (x – 640,00 zł), gdzie x oznacza cenę hurtową brutto leku, środka spożywczego specjalnego przeznaczenia żywieniowego albo wyrobu medycznego stanowiącego podstawę limitu, uwzględniającą liczbę DDD leku, jednostek środka spożywczego specjalnego przeznaczenia żywieniowego w opakowaniu albo liczbę jednostkowych wyrobów medycznych, albo liczbę jednostek wyrobu medycznego. Podmioty uprawnione do obrotu detalicznego, w rozumieniu ustawy z dnia 6 września 2001 r. – Prawo farmaceutyczne, są obowiązane stosować marżę, o której mowa w ust. 4. Dla leków wydawanych w trybie, o którym mowa w art. 43 ust. 1 pkt 2 i art. 46 ustawy o świadczeniach, dla których nie ustalono limitu finansowania, stosuje się urzędową marżę detaliczną, w wysokości określonej w ust. 4, liczoną od ceny hurtowej, i nie może być wyższa niż 20 zł. Dla leków oraz środków spożywczych specjalnego przeznaczenia żywieniowego, wydawanych w trybie, o którym mowa w art. 39 ust. 1, ustala się marżę hurtową w wysokości 10%. Marża detaliczna wynosi 100% wartości urzędowej marży detalicznej ustalonej w ust. 4 dla danego przedziału ceny hurtowej. Dla leków, o których mowa w art. 6 ust. 5, ustala się marżę detaliczną w wysokości 25% liczoną od kosztu jego sporządzenia, wynoszącą nie więcej niż koszt wykonania leku recepturowego przygotowywanego w warunkach aseptycznych, określony w przepisach wydanych na podstawie art. 6 ust. 10.",
        "output": PassageSplits(passage_splits=[
            "1. Ustala się urzędową marżę hurtową w wysokości 6% ceny zbytu netto leku, środka spożywczego specjalnego przeznaczenia żywieniowego lub wyrobu medycznego, nie niższą niż 50 groszy i nie wyższą niż 150 zł w przypadku, o którym mowa w art. 6 ust. 1 pkt 1 oraz 2000 zł w przypadkach, o których mowa w art. 6 ust. 1 pkt 2 i 3.",
            "2. Przedsiębiorcy prowadzący obrót hurtowy w rozumieniu art. 72 ust. 3 ustawy z dnia 6 września 2001 r. – Prawo farmaceutyczne są obowiązani stosować cenę zbytu netto wraz z marżą, o której mowa w ust. 1, oraz właściwą stawkę podatku od towarów i usług niezależnie od miejsca dostawy leku, środka spożywczego specjalnego przeznaczenia żywieniowego lub wyrobu medycznego będącego przedmiotem tego obrotu.",
            "3. Marża, o której mowa w ust. 1, może być dzielona pomiędzy przedsiębiorców prowadzących obrót hurtowy. Ustala się urzędową marżę detaliczną naliczaną od ceny hurtowej leku, środka spożywczego specjalnego przeznaczenia żywieniowego albo wyrobu medycznego stanowiącego podstawę limitu w danej grupie limitowej, w wysokości: od 5,00 zł do 40% marży, od 5,01 zł do 10,00 zł marża wynosi 2 zł + 30% ×(x −5,00 zł), od 10,01 zł do 20,00 zł marża wynosi 3,50 zł + 20% ×(x −10,00 zł), od 20,01 zł do 40,00 zł marża wynosi 5,50 zł + 15% ×(x −20,00 zł), od 40,01 zł do 80,00 zł marża wynosi 8,50 zł + 10% ×(x −40,00 zł), od 80,01 zł do 160,00 zł marża wynosi 12,50 zł + 5% ×(x −80,00 zł), od 160,01 zł do 320,00 zł marża wynosi 16,50 zł + 2,5% ×(x −160,00 zł), od 320,01 zł do 640,00 zł marża wynosi 20,50 zł + 2,5% ×(x −320,00 zł), od 640,01 zł do 1280,00 zł marża wynosi 28,50 zł + 2,5% ×(x −640,00 zł), a powyżej 1280,01 zł marża wynosi 44,50 zł + 1,25% ×(x −1280,00 zł), gdzie x oznacza cenę hurtową leku, środka spożywczego specjalnego przeznaczenia żywieniowego albo wyrobu medycznego stanowiącego podstawę limitu, uwzględniającą liczbę DDD leku, jednostek środka spożywczego specjalnego. przeznaczenia żywieniowego w opakow aniu albo liczbę jednostkowych wyrobów medycznych, albo liczbę jednostek wyrobu medycznego. Ustala się urzędową marżę detaliczną naliczaną od ceny hurtowej brutto leku, środka spożywczego specjalnego przeznaczenia żywieniowego albo wyrobu medycznego stanowiącego podstawę limitu w danej grupie limitowej, w wysokości: od 10,00 zł do 0,45 zł marża wynosi 35% * x, od 10,01 zł do 20,00 zł marża wynosi 3,95 zł + 25% * (x – 10,00 zł), od 20,01 zł do 40,00 zł marża wynosi 6,45 zł + 15% * (x – 20,00 zł), od 40,01 zł do 80,00 zł marża wynosi 9,45 zł + 10% * (x – 40,00 zł), od 80,01 zł do 160,00 zł marża wynosi 13,45 zł + 5% * (x – 80,00 zł), od 160,01 zł do 640,00 zł marża wynosi 17,45 zł + 2,75% * (x – 160,00 zł), a powyżej 640,01 zł marża wynosi 30,65 zł + 2% * (x – 640,00 zł), gdzie x oznacza cenę hurtową brutto leku, środka spożywczego specjalnego przeznaczenia żywieniowego albo wyrobu medycznego stanowiącego podstawę limitu, uwzględniającą liczbę DDD leku, jednostek środka spożywczego specjalnego przeznaczenia żywieniowego w opakowaniu albo liczbę jednostkowych wyrobów medycznych, albo liczbę jednostek wyrobu medycznego. Podmioty uprawnione do obrotu detalicznego, w rozumieniu ustawy z dnia 6 września 2001 r. – Prawo farmaceutyczne, są obowiązane stosować marżę, o której mowa w ust. 4. Dla leków wydawanych w trybie, o którym mowa w art. 43 ust. 1 pkt 2 i art. 46 ustawy o świadczeniach, dla których nie ustalono limitu finansowania, stosuje się urzędową marżę detaliczną, w wysokości określonej w ust. 4, liczoną od ceny hurtowej, i nie może być wyższa niż 20 zł. Dla leków oraz środków spożywczych specjalnego przeznaczenia żywieniowego, wydawanych w trybie, o którym mowa w art. 39 ust. 1, ustala się marżę hurtową w wysokości 10%. Marża detaliczna wynosi 100% wartości urzędowej marży detalicznej ustalonej w ust. 4 dla danego przedziału ceny hurtowej. Dla leków, o których mowa w art. 6 ust. 5, ustala się marżę detaliczną w wysokości 25% liczoną od kosztu jego sporządzenia, wynoszącą nie więcej niż koszt wykonania leku recepturowego przygotowywanego w warunkach aseptycznych, określony w przepisach wydanych na podstawie art. 6 ust. 10."])
    },
    {
        "input": "[1. W przypadku gdy zobowiązanie, o którym mowa w art. 25 pkt 4, w zakresie dotyczącym rocznej wiel kości dostaw lub ciągłości dostaw, nie zostani e dotrzymane i nastąpi niezaspokojenie potrzeb świadczeniobiorców, wnioskodawca, który otrzymał decyzję administracyjną o objęciu refundacją jest obowiązany do zwrotu do Funduszu kwoty stanowiącej iloczyn liczby niedostarczonych jednostkowych opakowań leku , środka spożywczego specjalnego przeznaczenia żywieniowego albo jednostkowych wyrobów medycznych i ich urzędowej ceny zbytu netto, chyba że niewykonywanie tego zobowiązania jest następstwem działania siły wyższej albo potrzeby świadczeniobiorców zostały z aspokojone przez jego odpowiednik. ] <1. W przypadku gdy zobowiązanie, o którym mowa w art. 11 ust. 2 pkt 9, w zakresie dotyczącym rocznej wielkości dostaw lub ciągłości dostaw nie zostanie dotrzymane, wnioskodawca, który otrzymał decyzję administracyjną o objęciu refundacją, jest obowiązany do zwrotu do Funduszu kwoty stanowiącej iloczyn liczby niedostarczonych jednostkowych opakowań leku, środka spożywczego specjalnego przeznaczenia żywieniowego albo jednostkowych wyrobów medycznych i ich urzędowej ceny zbytu, chyba że niewykonywanie tego zobowiązania jest następstwem działania siły wyższej albo wnioskodawca dopełnił obowiązku określonego w ust. 2.> <1a. Niezależnie od zobowiązania, o którym mowa w ust. 1, w przypadku gdy zobowiązanie, o którym mowa w art. 11 ust. 2 pkt 9, w zakresie dotyczącym rocznej wielkoś ci dostaw, z uwzględnieniem ujęcia miesięcznego, lub ciągłości dostaw nie zostanie dotrzymane, a świadczeniobiorca otrzyma lek sprowadzony w trybie określonym w art. 4 ust. 1, 8 lub 9 ustawy z dnia 6 września 2001 r. – Prawo farmaceutyczne, wnioskodawca, k tóry otrzymał decyzję administracyjną o objęciu refundacją, jest obowiązany do zwrotu do Funduszu kwoty stanowiącej różnicę między ustaloną urzędową ceną zbytu dla tego leku a faktycznie poniesionymi przez Fundusz wydatkami na refundację tego leku. 1b. Przepis ust. 1a stosuje się również do środka spożywczego specjalnego przeznaczenia żywieniowego oraz wyrobu medycznego. >",
        "output": PassageSplits(passage_splits=[
            "[1. W przypadku gdy zobowiązanie, o którym mowa w art. 25 pkt 4, w zakresie dotyczącym rocznej wiel kości dostaw lub ciągłości dostaw, nie zostani e dotrzymane i nastąpi niezaspokojenie potrzeb świadczeniobiorców, wnioskodawca, który otrzymał decyzję administracyjną o objęciu refundacją jest obowiązany do zwrotu do Funduszu kwoty stanowiącej iloczyn liczby niedostarczonych jednostkowych opakowań leku , środka spożywczego specjalnego przeznaczenia żywieniowego albo jednostkowych wyrobów medycznych i ich urzędowej ceny zbytu netto, chyba że niewykonywanie tego zobowiązania jest następstwem działania siły wyższej albo potrzeby świadczeniobiorców zostały z aspokojone przez jego odpowiednik. ]",
            "<1. W przypadku gdy zobowiązanie, o którym mowa w art. 11 ust. 2 pkt 9, w zakresie dotyczącym rocznej wielkości dostaw lub ciągłości dostaw nie zostanie dotrzymane, wnioskodawca, który otrzymał decyzję administracyjną o objęciu refundacją, jest obowiązany do zwrotu do Funduszu kwoty stanowiącej iloczyn liczby niedostarczonych jednostkowych opakowań leku, środka spożywczego specjalnego przeznaczenia żywieniowego albo jednostkowych wyrobów medycznych i ich urzędowej ceny zbytu, chyba że niewykonywanie tego zobowiązania jest następstwem działania siły wyższej albo wnioskodawca dopełnił obowiązku określonego w ust. 2.>",
            "<1a. Niezależnie od zobowiązania, o którym mowa w ust. 1, w przypadku gdy zobowiązanie, o którym mowa w art. 11 ust. 2 pkt 9, w zakresie dotyczącym rocznej wielkoś ci dostaw, z uwzględnieniem ujęcia miesięcznego, lub ciągłości dostaw nie zostanie dotrzymane, a świadczeniobiorca otrzyma lek sprowadzony w trybie określonym w art. 4 ust. 1, 8 lub 9 ustawy z dnia 6 września 2001 r. – Prawo farmaceutyczne, wnioskodawca, k tóry otrzymał decyzję administracyjną o objęciu refundacją, jest obowiązany do zwrotu do Funduszu kwoty stanowiącej różnicę między ustaloną urzędową ceną zbytu dla tego leku a faktycznie poniesionymi przez Fundusz wydatkami na refundację tego leku. 1b. Przepis ust. 1a stosuje się również do środka spożywczego specjalnego przeznaczenia żywieniowego oraz wyrobu medycznego. >"
        ])
    }
]

passage_split_examples_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}")
    ]
)

few_shot_passage_split_prompt = FewShotChatMessagePromptTemplate(
    examples=passage_split_examples,
    example_prompt=passage_split_examples_prompt
)

passage_split_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Please process the following text carefully. It is important to maintain the integrity of the content, including all punctuation, brackets, and special characters. Do not omit or alter any of the brackets or other characters. you got given article, seperate this text on each passage, don't split on subpoints such as x) [for instance 1) 2) 3) ... n)] !!!!!!!!, give it in array, don't split on subpoints!!, is so important don't split on 1), 2) ... n) !!!!!, don't split on subsections, only on passages"),
    few_shot_passage_split_prompt,
    ("user", "{input}")
])





In [15]:
from langchain_core.documents import Document

text = """   2) ilość leku recepturowego, którego dotyczy odpłatność ryczałtowa, oraz sposób
obliczania kosztu sporządzania leku recepturowego
– biorąc pod uwagę dostępność do leków, bezpieczeństwo ich stosowania oraz postać
farmaceutyczną.
Art. 7. 1. Ustala się urzędową marżę hurtową w wysokości 6% ceny zbytu netto
leku, środka spożywczego specjalnego przeznaczenia żywieniowego lub wyrobu
medycznego, nie niższą niż 50 groszy i nie wyższą niż:
1) 150 zł – w przypadku, o którym mowa w art. 6 ust. 1 pkt 1;
2) 2000 zł – w przypadkach, o których mowa w art. 6 ust. 1 pkt 2 i 3.
2. Przedsiębiorcy prowadzący obrót hurtowy w rozumieniu art. 72 ust. 3 ustawy
z dnia 6 września 2001 r. – Prawo farmaceutyczne są obowiązani stosować cenę zbytu
netto wraz z marżą, o której mowa w ust. 1, oraz właściwą stawkę podatku od towarów
i usług niezależnie od miejsca dostawy leku, środka spożywczego specjalnego
przeznaczenia żywieniowego lub wyrobu medycznego będącego przedmiotem tego
obrotu.
3. Marża, o której mowa w ust. 1, może być dzielona pomiędzy przedsiębiorców
prowadzących obrót hurtowy.
[4. Ustala się urzędową marżę detaliczną naliczaną od ceny hurtowej leku,
środka spożywczego specjalnego przeznaczenia żywieniowego albo wyrobu
medycznego stanowiącego podstawę limitu w danej grupie limitowej, w wysokości:
od do zasada marży
– 5,00 zł 40%
          5,01 zł 10,00 zł 2 zł + 30% × (x − 5,00 zł)
10,01 zł 20,00 zł 3,50 zł + 20% × (x −10,00 zł)
20,01 zł 40,00 zł 5,50 zł + 15% × (x −20,00 zł)
40,01 zł 80,00 zł 8,50 zł + 10% × (x −40,00 zł)
80,01 zł 160,00 zł 12,50 zł + 5% × (x −80,00 zł)
160,01 zł 320,00 zł 16,50 zł + 2,5% × (x − 160,00 zł)
320,01 zł 640,00 zł 20,50 zł + 2,5% × (x − 320,00 zł)
640,01 zł 1280,00 zł 28,50 zł + 2,5% × (x − 640,00 zł)
1280,01 zł 44,50 zł + 1,25% × (x − 1280,00 zł)
– gdzie x oznacza cenę hurtową leku, środka spożywczego specjalnego przeznaczenia
żywieniowego albo wyrobu medycznego stanowiącego podstawę limitu,
uwzględniającą liczbę DDD leku, jednostek środka spożywczego specjalnego"""

table_chain.invoke({"input": text})

{'plain_text': '2) ilość leku recepturowego, którego dotyczy odpłatność ryczałtowa, oraz sposób obliczania kosztu sporządzania leku recepturowego – biorąc pod uwagę dostępność do leków, bezpieczeństwo ich stosowania oraz postać farmaceutyczną. Art. 7. 1. Ustala się urzędową marżę hurtową w wysokości 6% ceny zbytu netto leku, środka spożywczego specjalnego przeznaczenia żywieniowego lub wyrobu medycznego, nie niższą niż 50 groszy i nie wyższą niż 150 zł w przypadku, o którym mowa w art. 6 ust. 1 pkt 1 oraz 2000 zł w przypadkach, o których mowa w art. 6 ust. 1 pkt 2 i 3. 2. Przedsiębiorcy prowadzący obrót hurtowy w rozumieniu art. 72 ust. 3 ustawy z dnia 6 września 2001 r. – Prawo farmaceutyczne są obowiązani stosować cenę zbytu netto wraz z marżą, o której mowa w ust. 1, oraz właściwą stawkę podatku od towarów i usług niezależnie od miejsca dostawy leku, środka spożywczego specjalnego przeznaczenia żywieniowego lub wyrobu medycznego będącego przedmiotem tego obrotu. 3. Marża, o której m

In [20]:
print(response[0].metadata)


{'plain_text': '2) ilość leku recepturowego, którego dotyczy odpłatność ryczałtowa, oraz sposób obliczania kosztu sporządzania leku recepturowego – biorąc pod uwagę dostępność do leków, bezpieczeństwo ich stosowania oraz postać farmaceutyczną. Art. 7. 1. Ustala się urzędową marżę hurtową w wysokości 6% ceny zbytu netto leku, środka spożywczego specjalnego przeznaczenia żywieniowego lub wyrobu medycznego, nie niższą niż 50 groszy i nie wyższą niż: 1) 150 zł – w przypadku, o którym mowa w art. 6 ust. 1 pkt 1; 2) 2000 zł – w przypadkach, o których mowa w art. 6 ust. 1 pkt 2 i 3. 2. Przedsiębiorcy prowadzący obrót hurtowy w rozumieniu art. 72 ust. 3 ustawy z dnia 6 września 2001 r. – Prawo farmaceutyczne są obowiązani stosować cenę zbytu netto wraz z marżą, o której mowa w ust. 1, oraz właściwą stawkę podatku od towarów i usług niezależnie od miejsca dostawy leku, środka spożywczego specjalnego przeznaczenia żywieniowego lub wyrobu medycznego będącego przedmiotem tego obrotu. 3. Marża, o k

In [ ]:
# gpt-4o-mini

In [9]:

import types


def calculate_retail_margin(x):
    pass


try:
    exec('''def oblicz_marze_detaliczna(cena_hurtowa):
    """
    Oblicza urzędową marżę detaliczną na podstawie ceny hurtowej leku, środka spożywczego specjalnego przeznaczenia żywieniowego albo wyrobu medycznego.

    Argumenty:
    cena_hurtowa (float): Cena hurtowa w złotych.

    Zwraca:
    float: Obliczona marża detaliczna w złotych.

    Uwagi:
    Funkcja obsługuje różne przedziały cenowe zgodnie z dostarczoną tabelą marży detalicznej.
    """
    if cena_hurtowa <= 5.00:
        marza = 0.40 * cena_hurtowa
    elif 5.01 <= cena_hurtowa <= 10.00:
        marza = 2 + 0.30 * (cena_hurtowa - 5.00)
    elif 10.01 <= cena_hurtowa <= 20.00:
        marza = 3.50 + 0.20 * (cena_hurtowa - 10.00)
    elif 20.01 <= cena_hurtowa <= 40.00:
        marza = 5.50 + 0.15 * (cena_hurtowa - 20.00)
    elif 40.01 <= cena_hurtowa <= 80.00:
        marza = 8.50 + 0.10 * (cena_hurtowa - 40.00)
    elif 80.01 <= cena_hurtowa <= 160.00:
        marza = 12.50 + 0.05 * (cena_hurtowa - 80.00)
    elif 160.01 <= cena_hurtowa <= 320.00:
        marza = 16.50 + 0.025 * (cena_hurtowa - 160.00)
    elif 320.01 <= cena_hurtowa <= 640.00:
        marza = 20.50 + 0.025 * (cena_hurtowa - 320.00)
    elif 640.01 <= cena_hurtowa <= 1280.00:
        marza = 28.50 + 0.025 * (cena_hurtowa - 640.00)
    else:
        marza = 44.50 + 0.0125 * (cena_hurtowa - 1280.00)
    return marza
    ''')
except SyntaxError as e:
    print(f"SyntaxError: {e}")
except Exception as e:
    print(f"Error: {e}")
function_name = "oblicz_marze_detaliczna"
calculate_margin = locals()[function_name]
#
# price = 750.00
# margin2 = calculate_retail_margin(price)

calculate_retail_margin = tool(calculate_margin)

# structured_tool = StructuredTool.from_function(func=calculate_retail_margin)

# print(structured_tool.invoke({"x": price}))
#
# tools = []
#
# tools.append(calculate_retail_margin)
#
# print(calculate_retail_margin)
# print(f"The margin for {price} zł is {margin2} zł.")


In [12]:
os.environ["OPENAI_API_KEY"] = OPENAI_KEY["OPEN_API_KEY"]

In [26]:
!pip install --upgrade langchain


     ---------------------------------------- 0.0/997.8 kB ? eta -:--:--
     - ------------------------------------- 30.7/997.8 kB 1.4 MB/s eta 0:00:01
     --- --------------------------------- 81.9/997.8 kB 919.0 kB/s eta 0:00:01
     ----- -------------------------------- 153.6/997.8 kB 1.1 MB/s eta 0:00:01
     -------- ----------------------------- 235.5/997.8 kB 1.4 MB/s eta 0:00:01
     ------------------ ------------------- 491.5/997.8 kB 2.4 MB/s eta 0:00:01
     --------------------------------- ---- 890.9/997.8 kB 3.3 MB/s eta 0:00:01
     -------------------------------------  993.3/997.8 kB 3.2 MB/s eta 0:00:01
     -------------------------------------- 997.8/997.8 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/393.9 kB ? eta -:--:--
     ------------------------------------- 393.9/393.9 kB 24.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.22
    Uninstalling langchain-core-0.2.22:


In [21]:
os.environ["OPENAI_API_KEY"] = OPENAI_KEY["OPEN_API_KEY"]

In [12]:


tools = [calculate_retail_margin]
# agent = OpenAIAssistantV2Runnable.create_assistant(
#     name="langchain assistant",
#     instructions="Jesteś doradcą prawnym który ma odpowiadać na zadane pytania używając otrzymane narzędzia funkcje",
#     tools=tools,
#     model="gpt-4o",
# )
model4o = ChatOpenAI(model="gpt-4o-mini")
prompt4o = ChatPromptTemplate.from_messages(
    [
        ("system",
         "Jesteś doradcą prawnym który ma odpowiadać na zadane pytania używając otrzymane narzędzia funkcje, zwróc uwagę na poprawność odpowiedzi i zastosowanie polskich znaków diakrytycznych"),
        ("human", "{input}"),
        # # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)
agent = create_tool_calling_agent(model4o, tools, prompt4o)



agent_executor = AgentExecutor(agent=agent, tools=tools)
kk = agent_executor.invoke(
    {"input": "Jaką marżę detaliczną należy zastosować dla leku, którego cena hurtowa wynosi 750 zł?"})
# kk = agent_executor.invoke({"input": "ile jest 2+2?"})

kk

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [68]:
kk

{'input': 'Jaką marżę detaliczną należy zastosować dla leku, którego cena hurtowa wynosi 750 zł?',
 'output': 'Dla leku, którego cena hurtowa wynosi 750 zł, należy zastosować marżę detaliczną w wysokości 31,25 zł.'}

In [14]:
class LLMChainCallError(Exception):
    pass

class LLMChain:
    def __init__(self, model:BaseChatModel, output_schema, system_massage:Tuple, user_massage:Tuple, system_massage_retry:Tuple=None, few_shot_examples:list=None):
        self.chain = self._build_chain(model, output_schema, system_massage, user_massage, few_shot_examples)
        self.retry_chain = self._build_chain(model, output_schema, system_massage_retry, user_massage, few_shot_examples) if system_massage_retry else None
        l = 4


    def invoke_chain(self, input:Input, retry:bool=False):
        return self.retry_chain.invoke(input) if retry and self.retry_chain else self.chain.invoke(input)

    def call_chain(self, input:Input, retry:bool=False):
        try:
            response = self.invoke_chain(input, retry)

            return response
        except (openai.error.InvalidRequestError, openai.error.AuthenticationError) as e:
            # Handle invalid request error or authentication error
            raise LLMChainCallError(f"Invalid request or authentication error: {e}, during call {input}")
        except (openai.error.APIConnectionError, openai.error.RateLimitError) as e:
            # Handle API connection error or rate limit error
            raise LLMChainCallError(f"API connection or rate limit error: {e}, during call {input}")
        except Exception as e:
            # Handle any other unexpected error
            raise LLMChainCallError(f"An unexpected error occurred: {e}, during call {input}")

    def _get_prompt_examples(self, few_shot_examples: list):
        prompt_examples_form = ChatPromptTemplate.from_messages(
            [
                ("human", "{input}"),
                ("ai", "{output}")
            ]
        )
        return FewShotChatMessagePromptTemplate(
            examples=few_shot_examples,
            example_prompt=prompt_examples_form
        )

    def _get_prompt(self, system_massage:Tuple, user_massage:Tuple, few_shot_examples:list=None):
        if few_shot_examples:
           return ChatPromptTemplate.from_messages([
               system_massage,
               self._get_prompt_examples(few_shot_examples),
               user_massage
           ])
        else:
            return ChatPromptTemplate.from_messages([
                system_massage,
                user_massage
            ])

    def _build_chain(self,model:BaseChatModel, output_schema, system_massage:Tuple, user_massage:Tuple, few_shot_examples:list=None):
        return self._get_prompt(system_massage=system_massage,
                                user_massage= user_massage,
                                few_shot_examples=few_shot_examples) \
            | model.with_structured_output(output_schema)


In [30]:


class TableUtils(ABC):


    @staticmethod
    def create_function_from_table(llmchain:LLMChain, input:Input, retry:int=1):
        table_function = llmchain.call_chain(input, retry!=1)
        try:
            exec(table_function.function_definition)
            function_defined = locals()[table_function.function_name]
            checked_could_define_function = tool(function_defined)
            return table_function
        except Exception as e:
            print(f"Error: {e}")
            if retry > 3:
                return None
            else :
                print(f"It retried to create function from table, retry:{retry}")
                return TableUtils.create_function_from_table(llmchain, input, retry=retry+1)

    @staticmethod
    def summarize_table_and_erase_it_from_passage(llmchain:LLMChain, input:Input, retry:int=1):
        table_start_pattern = r'\/\s*t\s*a\s*b\s*l\s*e\s*_\s*s\s*t\s*a\s*r\s*t\s*\/'


        legal_act_with_erased_table = llmchain.call_chain(input, retry!=1)
        try:
            if re.search(table_start_pattern, legal_act_with_erased_table.legal_act_part_text) and len(legal_act_with_erased_table.table_summarization) > 0 :
                text_with_erased_table_start_annotation = re.sub(table_start_pattern, f"/{legal_act_with_erased_table.table_summarization}/", legal_act_with_erased_table.legal_act_part_text)
                return text_with_erased_table_start_annotation
            else:
                raise ValueError("Table not found in the text or table summarization is empty.")
        except Exception as e:
            print(f"Error: {e}")
            if retry > 3:
                return None
            else:
                print(f"It retried to summarize table and erase it from passage, retry:{retry}")
                return TableUtils.summarize_table_and_erase_it_from_passage(llmchain, input, retry=retry+1)

    @staticmethod
    def add_annotation_of_table(llmchain:LLMChain, input:Input, retry: int = 1):
        annotation_of_table = llmchain.call_chain(input, retry!=1)
        try:
            if annotation_of_table is None or "/table_start/" not in annotation_of_table.text_page or \
                    annotation_of_table.text_page.strip().endswith("/table_start/") or \
                    annotation_of_table.text_page.strip().startswith("/table_start/"):
                raise ValueError("Table annotation not found in the text or it is incorrect.")
            else:
                return annotation_of_table.text_page
        except Exception as e:
            print(f"Error: {e}")
            if retry > 3:
                return ValueError(f"Failed to add annotation of table text {input} after {retry-1} retries.")
            else:
                print(f"It retried to add annotation of table text, retry:{retry}")
                return TableUtils.add_annotation_of_table(llmchain, input, retry=retry+1)



In [23]:
class CreateAnnotationOfTable(BaseModel):
    text_page: str

table_annotation_examples = [
    {
        "input": '''
        opakowaniu albo liczbę jednostkowych wyrobów medycznych, albo liczbę jednostek wyrobu medycznego.]
<4. Ustala się urzędową marżę detaliczną naliczaną od ceny hurtowej brutto Nowe brzmienie leku, środka spożywczego specjalnego przeznaczenia żywieniowego albo wyrobu ust. 4 w art. 7 wejdzie w życie z dn. 1.01.2025 r. (Dz. U. z 2023 r. poz. 1938).
od do zasada marży
– 10,00 zł 0,45 zł + 35% * x
10,01 zł 20,00 zł 3,95 zł + 25% * (x – 10,00 zł)
20,01 zł 40,00 zł 6,45 zł + 15% * (x – 20,00 zł)
40,01 zł 80,00 zł 9,45 zł + 10% * (x – 40,00 zł)
80,01 zł 160,00 zł 13,45 zł + 5% * (x – 80,00 zł)
160,01 zł 640,00 zł 17,45 zł + 2,75% * (x – 160,00 zł)
640,01 zł 30,65 zł + 2% * (x – 640,00 zł)
– gdzie x oznacza cenę hurtową brutto leku, środka spożywczego specjalnego przeznaczenia żywieniowego albo wyrobu medycznego stanowiącego podstawę limitu, uwzględniającą liczbę DDD leku, jednostek środka spożywczego specjalnego przeznaczenia żywieniowego w''',
        "output": CreateAnnotationOfTable(text_page='''
        opakowaniu albo liczbę jednostkowych wyrobów medycznych, albo liczbę jednostek wyrobu medycznego.]
<4. Ustala się urzędową marżę detaliczną naliczaną od ceny hurtowej brutto Nowe brzmienie leku, środka spożywczego specjalnego przeznaczenia żywieniowego albo wyrobu ust. 4 w art. 7 wejdzie w życie z dn. 1.01.2025 r. (Dz. U. z 2023 r. poz. 1938).
/table_start/
od do zasada marży
– 10,00 zł 0,45 zł + 35% * x
10,01 zł 20,00 zł 3,95 zł + 25% * (x – 10,00 zł)
20,01 zł 40,00 zł 6,45 zł + 15% * (x – 20,00 zł)
40,01 zł 80,00 zł 9,45 zł + 10% * (x – 40,00 zł)
80,01 zł 160,00 zł 13,45 zł + 5% * (x – 80,00 zł)
160,01 zł 640,00 zł 17,45 zł + 2,75% * (x – 160,00 zł)
640,01 zł 30,65 zł + 2% * (x – 640,00 zł)
– gdzie x oznacza cenę hurtową brutto leku, środka spożywczego specjalnego przeznaczenia żywieniowego albo wyrobu medycznego stanowiącego podstawę limitu, uwzględniającą liczbę DDD leku, jednostek środka spożywczego specjalnego przeznaczenia żywieniowego w''')
    },
    {
        "input": '''
        niniejszej ustawy, pomniejszona o marżę hurtową w wysokości 8,91% liczoną od ceny urzędowej hurtowej oraz marżę detaliczną w wysokości:
Cena hurtowa w złotych Marża detaliczna liczona od ceny hurtowej
0–3,60 40%
3,61–4,80 1,44 zł
4,81–6,50 30%
6,51–9,75 1,95 zł
9,76–14,00 20%
14,01–15,55 2,80 zł
15,56–30,00 18%
30,01–33,75 5,40 zł
33,76–50,00 16%
50,01–66,67 8,00 zł
66,68–100,00 12%
powyżej 100,00 12,00 zł
Negocjacje przeprowadza Komisja. Do negocjacji stosuje się art. 18 ust. 3, art. 19 ust. 1 i 2 pkt 2–7, art. 20 i art. 22 oraz przepisy wykonawcze wydane na podstawie art. 23.
        ''',
        "output": CreateAnnotationOfTable(text_page='''
        niniejszej ustawy, pomniejszona o marżę hurtową w wysokości 8,91% liczoną od ceny urzędowej hurtowej oraz marżę detaliczną w wysokości:
/table_start/
Cena hurtowa w złotych Marża detaliczna liczona od ceny hurtowej
0–3,60 40%
3,61–4,80 1,44 zł
4,81–6,50 30%
6,51–9,75 1,95 zł
9,76–14,00 20%
14,01–15,55 2,80 zł
15,56–30,00 18%
30,01–33,75 5,40 zł
33,76–50,00 16%
50,01–66,67 8,00 zł
66,68–100,00 12%
powyżej 100,00 12,00 zł
Negocjacje przeprowadza Komisja. Do negocjacji stosuje się art. 18 ust. 3, art. 19 ust. 1 i 2 pkt 2–7, art. 20 i art. 22 oraz przepisy wykonawcze wydane na podstawie art. 23.
        ''')
    }
]

add_annotation_of_table_chain = LLMChain(
    model=openAIModel4o,
    output_schema=CreateAnnotationOfTable,
    system_massage=("system",
                    """"
                     You will get document page with text and/or with table/ tables, where table start add annotation /table_start/ just this not any other annotation, there are one or more tables, beside that keep text not changed but just add this annotation, you cannot remove any signs!!!!!!
                    """ ),
    system_massage_retry=("system",
                          """"
                           You will get document page with text and/or with table/ tables, where table start add annotation /table_start/ just this not any other annotation, there are one or more tables, beside that keep text not changed but just add this annotation, you cannot remove any signs!!!!!!

                            You previously placed incorrectly /table start/, do it again"""),
    user_massage=("user", "{input}"),
    few_shot_examples=table_annotation_examples

)

In [24]:
class LegalActWithErasedTableWithAnnotation(BaseModel):
    '''
        Model zawierający tekst aktu prawnego z usuniętą tabelą oraz adnotację dotyczącą usuniętej tabeli.

        - `legal_act_part_text`: Oryginalny tekst aktu prawnego, z którego usunięto tabelę.
        - `table_summarization`: Opis ogólny zawartości tabeli, bez informacji szczegółowych.
    '''
    legal_act_part_text: str
    table_summarization: str

table_erasing_examples = [
    {
        "input": '''
        opakowaniu albo liczbę jednostkowych wyrobów medycznych, albo liczbę jednostek wyrobu medycznego.]
<4. Ustala się urzędową marżę detaliczną naliczaną od ceny hurtowej brutto Nowe brzmienie leku, środka spożywczego specjalnego przeznaczenia żywieniowego albo wyrobu ust. 4 w art. 7
/table_start/
od do zasada marży
– 10,00 zł 0,45 zł + 35% * x
10,01 zł 20,00 zł 3,95 zł + 25% * (x – 10,00 zł)
20,01 zł 40,00 zł 6,45 zł + 15% * (x – 20,00 zł)
40,01 zł 80,00 zł 9,45 zł + 10% * (x – 40,00 zł)
80,01 zł 160,00 zł 13,45 zł + 5% * (x – 80,00 zł)
160,01 zł 640,00 zł 17,45 zł + 2,75% * (x – 160,00 zł)
640,01 zł 30,65 zł + 2% * (x – 640,00 zł)
– gdzie x oznacza cenę hurtową brutto leku, środka spożywczego specjalnego przeznaczenia żywieniowego albo wyrobu medycznego stanowiącego podstawę limitu, uwzględniającą liczbę DDD leku, jednostek środka spożywczego specjalnego przeznaczenia żywieniowego w''',
        "output": LegalActWithErasedTableWithAnnotation(legal_act_part_text='''
        opakowaniu albo liczbę jednostkowych wyrobów medycznych, albo liczbę jednostek wyrobu medycznego.]
<4. Ustala się urzędową marżę detaliczną naliczaną od ceny hurtowej brutto Nowe brzmienie leku, środka spożywczego specjalnego przeznaczenia żywieniowego albo wyrobu ust. 4 w art. 7
/table_start/
'''
                                                        , table_summarization='''
Tabela opisuje zasady naliczania marży detalicznej w zależności od ceny hurtowej produktu. Marża składa się z kwoty bazowej oraz procentowej wartości, która zależy od przedziału cenowego. Im wyższa cena produktu, tym niższy procent naliczanej marży.
'''
                                                        )
    },
    {
        "input": '''
        niniejszej ustawy, pomniejszona o marżę hurtową w wysokości 8,91% liczoną od ceny urzędowej hurtowej oraz marżę detaliczną w wysokości:
        /table_start/
Cena hurtowa w złotych Marża detaliczna liczona od ceny hurtowej
0–3,60 40%
3,61–4,80 1,44 zł
4,81–6,50 30%
6,51–9,75 1,95 zł
9,76–14,00 20%
14,01–15,55 2,80 zł
15,56–30,00 18%
30,01–33,75 5,40 zł
33,76–50,00 16%
50,01–66,67 8,00 zł
66,68–100,00 12%
powyżej 100,00 12,00 zł
Negocjacje przeprowadza Komisja. Do negocjacji stosuje się art. 18 ust. 3, art. 19 ust. 1 i 2 pkt 2–7, art. 20 i art. 22 oraz przepisy wykonawcze wydane na podstawie art. 23.
        ''',
        "output": LegalActWithErasedTableWithAnnotation(legal_act_part_text='''
        niniejszej ustawy, pomniejszona o marżę hurtową w wysokości 8,91% liczoną od ceny urzędowej hurtowej oraz marżę detaliczną w wysokości:
        /table_start/
Negocjacje przeprowadza Komisja. Do negocjacji stosuje się art. 18 ust. 3, art. 19 ust. 1 i 2 pkt 2–7, art. 20 i art. 22 oraz przepisy wykonawcze wydane na podstawie art. 23.
'''
                                                        , table_summarization='''
Tabela przedstawia zasady naliczania marży detalicznej w zależności od ceny hurtowej produktu. W niektórych przedziałach cenowych marża jest ustalana procentowo, a w innych jako stała kwota. Wyższe ceny hurtowe mają niższy procent marży lub ustaloną kwotę marży.
'''
                                                        )
    }
]


In [21]:
class AnnotationOfChanges(BaseModel):
    """Schema for extracting annotations of changes from legal texts"""

    arts: list[Art] = Field(description="List of articles with their respective ust and pkt")
    method: MethodEnum
    date: str


class AnnotationResponse(BaseModel):
    annotation: AnnotationOfChanges
    cleared_input: str = Field(
        description="Input with removed words, characters from start which not refer to change, add or remove of change, use polish characters such as ż, ź, ą, ę  e.t.c. don't omit them !!!")

annotation_examples = [
    {
        "input": "jego wytworzenia. Dodane ust. 1a i  1b w art. 34  wejdą w życie z  dn. 1.07.2024 r.  (Dz. U. z 2023 r.  poz. 1938).",
        "output": AnnotationResponse(
            annotation=AnnotationOfChanges(
                arts=[
                    Art(
                        art="34",
                        usts=[
                            Ust(ust="1a", pkt=[]),
                            Ust(ust="1b", pkt=[])
                        ]
                    )
                ],
                method=MethodEnum.add,
                date="1.07.2024"
            ),
            cleared_input="Dodane ust. 1a i  1b w art. 34  wejdą w życie z  dn. 1.07.2024 r.  (Dz. U. z 2023 r.  poz. 1938)."
        ).json()
    },
    {
        "input": "dr. Nowe brzmienie ust. 4 w art. 7 wejdzie w życie z dn. 1.01.2025 r. (Dz. U. z 2023 r. poz. 1938).       ",
        "output": AnnotationResponse(
            annotation=AnnotationOfChanges(
                arts=[
                    Art(
                        art="7",
                        usts=[
                            Ust(ust="4", pkt=[])
                        ]
                    )
                ],
                method=MethodEnum.change,
                date="1.01.2025"
            ),
            cleared_input="Nowe brzmienie ust. 4 w art. 7 wejdzie w życie z dn. 1.01.2025 r. (Dz. U. z 2023 r. poz. 1938)."
        ).json()
    }
]


annotations_of_changes_chain = LLMChain(
    model=openAIModel,
    output_schema=AnnotationResponse,
    system_massage=("system",
                    "Extract the desired information from the following passage.  Only extract the properties mentioned in the 'information_extraction' function.  Don't use \n, \t, or any other special characters, pay attention on cleared_input, because I need it erase the given sequence from legal text.  Don't add your formatting!!!!! leave it as it is in the input. Remember that user input use polish characters such as ż, ź e.t.c."),
    user_massage=("user", "{input}"),
    few_shot_examples=annotation_examples
)

In [22]:

class AnnotationRetryResponse(BaseModel):
    cleared_input: str

get_annotation_retry_for_removing_chain = LLMChain(
    model=model4o,
    output_schema=AnnotationRetryResponse,
    system_massage=("system",
                    """"
                     correct the following passage with using polish characters such as ą, ę, ć, ż, ź e.t.c
                    """),
    user_massage=("user", "{input}")
)

NameError: name 'model4o' is not defined

In [51]:
import inspect


print(inspect.getsource(l.func))

OSError: could not get source code

In [32]:
from functools import partial

# Define a function with multiple parameters
def my_function(x, y, z):
    return x + y + z

# Use partial to "pre-load" the first two arguments
partial_func = partial(my_function, 10, 20)

# Define another function that takes a pre-loaded function
def apply_preloaded_function(func, z_value):
    # Call the function with the remaining argument
    return func(z_value)

# Call the pre-loaded function by only providing the remaining argument
result = apply_preloaded_function(partial_func, 5)
print(result)  # Output: 35 (10 + 20 + 5)

35


In [25]:
from pydantic import BaseModel
from openai import OpenAI
from langchain_core.tools import tool

os.environ["OPENAI_API_KEY"] = OPENAI_KEY["OPEN_API_KEY"]
client = OpenAI()




class DefinitionFunctionCallJson(BaseModel):
    '''
        function_definition_json should be in this format:
                {
                    "name": ...,
                    "description": ...,
                    "parameters": ...
                }
    '''
    function_definition_json: dict







text_for_test = '''
2) ilość leku recepturowego, którego dotyczy odpłatność ryczałtowa, oraz sposób
obliczania kosztu sporządzania leku recepturowego
– biorąc pod uwagę dostępność do leków, bezpieczeństwo ich stosowania oraz postać
farmaceutyczną.
Art. 7. 1. Ustala się urzędową marżę hurtową w  wysokości 6% ceny zbytu netto
leku, środka spożywczego specjalnego przeznaczenia żywieniowego lub wyrobu
medycznego, nie niższą niż 50  groszy i  nie wyższą niż:
1) 150 zł – w przypadku, o  którym mowa w  art. 6 ust. 1 pkt 1;
2) 2000  zł – w przypadkach, o  których mowa w  art. 6 ust. 1 pkt 2 i 3.
2. Przedsiębiorcy prowadzący obrót hurtowy w  rozumieniu art.  72 ust. 3 ustawy
z dnia 6  września 2001  r. – Prawo farmaceutyczne są obowiązani stosować cenę zbytu
netto wraz z  marżą, o  której mowa w  ust. 1, oraz właściwą stawkę podatku od towarów
i usług niezależnie od miejsca dostawy leku, środka spożywczego specjalnego
przeznaczenia żywieniowego lub wyrobu medycznego będącego przedmiotem tego
obrotu.
3. Marża, o  której mowa w  ust. 1, może  być dzielona pomiędzy przedsiębiorców
prowadzących obrót hurtowy.
 [4. Ustala się urzędową marżę detaliczną naliczaną od ceny hurtowej leku,
środka spożywczego specjalnego przeznaczenia żywieniowego albo wyrobu
medycznego stanowiącego podstawę limitu w danej grupie limitowej, w  wysokości:
od do zasada marży
– 5,00 zł 40%
5,01 zł 10,00  zł 2 zł +30%  ×(x −5,00 zł)
10,01  zł 20,00  zł 3,50 zł +20%  ×(x −10,00 zł)
20,01  zł 40,00  zł 5,50 zł +15%  ×(x −20,00 zł)
40,01  zł 80,00  zł 8,50 zł +10%  ×(x −40,00 zł)
80,01  zł 160,00  zł 12,50 zł +5% ×(x −80,00 zł)
160,01  zł 320,00  zł 16,50 zł +2,5% ×(x −160 ,00 zł)
320,01  zł 640,00  zł 20,50 zł +2,5% ×(x −320 ,00 zł)
640,01  zł 1280,00  zł 28,50 zł +2,5% ×(x −640 ,00 zł)
1280,01  zł  44,50 zł +1,25%  ×(x −1280 ,00 zł)
– gdzie x oznacza cenę hurtową leku, środka spożywczego specjalnego przeznaczenia
żywieniowego albo wyrobu medycznego stanowiącego podstawę limitu,
uwzględniającą liczbę DDD leku, jednostek środka spożywczego specjalnego

'''

model4o = ChatOpenAI(model="gpt-4o")


prompt_for_getting_json_function_definition = ChatPromptTemplate.from_messages(
    [
        ("system", '''
            convert it openai json format function definition
        '''),
        ("user", "{input}"),
    ]
)

get_json_function_definition_chain = prompt_for_getting_json_function_definition | model4o.with_structured_output(
    DefinitionFunctionCallJson)

l2 = get_json_function_definition_chain.invoke({"input": l['function_definition']})

# completion = client.chat.completions.create(
#     model="gpt-4o",
#     messages=[
#         {"role": "system", "content": '''
#         look at this text, there is a table, create function that it will handle this table and define name of this function and arguments which it will get
#             1. Ensure the function handles all specifications in table
#             2. comment briefly inside function will all necessary information from given text related to this function
#             3 use only polish language
#             4. return function in python but give it in format
#
#         '''},
#         {"role": "user", "content": f"{text_for_test}"},
#     ],
#     response_format=DefinitionTableFunction,
# )
#
#
# completion.choices[0].message.parse
t = 4

NameError: name 'l' is not defined

In [27]:
!pip install openai

In [46]:
import openai


def calculate_retail_margin(x):
    """
    This function calculates the official retail margin based on the wholesale price (x) of a medication,
    special nutritional food, or medical device. The margin is determined according to the specified
    price ranges in the table provided by the legal text.

    Arguments:
    x : float
        The wholesale price of the item.

    Returns:
    float
        The calculated retail margin based on the price range.
    """

    if x <= 5.00:
        # For prices up to 5.00 zł, the margin is 40% of the price.
        return 0.40 * x
    elif 5.01 <= x <= 10.00:
        # For prices between 5.01 zł and 10.00 zł, the margin is 2 zł + 30% of (x - 5.00 zł).
        return 2 + 0.30 * (x - 5.00)
    elif 10.01 <= x <= 20.00:
        # For prices between 10.01 zł and 20.00 zł, the margin is 3.50 zł + 20% of (x - 10.00 zł).
        return 3.50 + 0.20 * (x - 10.00)
    elif 20.01 <= x <= 40.00:
        # For prices between 20.01 zł and 40.00 zł, the margin is 5.50 zł + 15% of (x - 20.00 zł).
        return 5.50 + 0.15 * (x - 20.00)
    elif 40.01 <= x <= 80.00:
        # For prices between 40.01 zł and 80.00 zł, the margin is 8.50 zł + 10% of (x - 40.00 zł).
        return 8.50 + 0.10 * (x - 40.00)
    elif 80.01 <= x <= 160.00:
        # For prices between 80.01 zł and 160.00 zł, the margin is 12.50 zł + 5% of (x - 80.00 zł).
        return 12.50 + 0.05 * (x - 80.00)
    elif 160.01 <= x <= 320.00:
        # For prices between 160.01 zł and 320.00 zł, the margin is 16.50 zł + 2.5% of (x - 160.00 zł).
        return 16.50 + 0.025 * (x - 160.00)
    elif 320.01 <= x <= 640.00:
        # For prices between 320.01 zł and 640.00 zł, the margin is 20.50 zł + 2.5% of (x - 320.00 zł).
        return 20.50 + 0.025 * (x - 320.00)
    elif 640.01 <= x <= 1280.00:
        # For prices between 640.01 zł and 1280.00 zł, the margin is 28.50 zł + 2.5% of (x - 640.00 zł).
        return 28.50 + 0.025 * (x - 640.00)
    else:  # x > 1280.00
        # For prices above 1280.00 zł, the margin is 44.50 zł + 1.25% of (x - 1280.00 zł).
        return 44.50 + 0.0125 * (x - 1280.00)


tools = [
    {
        "type": "function",
        "function": {
            "name": "calculate_retail_margin",
            "description": "This function calculates the official retail margin based on the wholesale price (x) of a medication, special nutritional food, or medical device. The margin is determined according to the specified price ranges in the table provided by the legal text.",
            "parameters": {
                "type": "object",
                "properties": {
                    "x": {
                        "type": "number",
                        "description": "The wholesale price of the item."
                    }
                },
                "required": ["x"],
                "additionalProperties": False

            }
        }
    }
]

messages = [
    {"role": "system",
     "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."},
    {"role": "user", "content": "Jaka będzie marża detaliczna dla leku, którego cena hurtowa wynosi 75 zł?"}
]
print(OPENAI_KEY["OPEN_API_KEY"])

os.environ["OPENAI_API_KEY"] = OPENAI_KEY["OPEN_API_KEY"]

response = openai.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

r = 4


sk-proj-43WipF94bBmSt0O2ZdWmT3BlbkFJBe8PXpIiiRW1vkmHckZI


In [26]:


class CreateAnnotationOfTable(BaseModel):
    text_page: str


table_create_annotation_prompt = ChatPromptTemplate.from_messages([
    ("system",
     """"
      You will get document page with text and/or with table/ tables, where table start add annotation /table_start/, there are one or more tables, beside that keep text not changed but just add this annotation, you cannot remove any signs!!!!!!
     """),
    ("user", "{input}")
])

In [27]:


table_annotation_examples_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}")
    ]
)

few_shot_table_annotation_prompt = FewShotChatMessagePromptTemplate(
    examples=table_annotation_examples,
    example_prompt=table_annotation_examples_prompt
)

In [32]:
import openai


class TableAnnotation:
    def __init__(self, pages_numbers: list, pages: list):
        self.pages_numbers = pages_numbers
        self.pages = pages


class LegalActTextPreprocessor:
    def __init__(self):
        pass



def extract_tables_from_pdf(pdf_path):
    def is_current_table(current_table_pages_numbers):
        return len(current_table_pages_numbers) > 0

    all_tables = []
    current_table_pages_numbers = []
    current_table_pages_text = []
    last_table_page = -1
    data = []
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            tables = page.extract_tables()
            page_text = page.extract_text()

            new_page_text = re.sub("©\s*Kancelaria\s+Sejmu\s*s\s*\.\s*\d+\s*/\s*\d+\s*", '', page_text).replace("\n",
                                                                                                                " ")
            for table_num, table in enumerate(tables):
                k = str(table[0][0]).replace('\n', ' ')

                table_start_pos = new_page_text.find(str(table[0][0]).replace('\n', ' ')[:20])

                if page_num == last_table_page + 1 and table_num == 0:
                    # Check for intervening text
                    intervening_text = new_page_text[:table_start_pos].strip()
                    if intervening_text:
                        # If there's intervening text, it's a new table
                        if is_current_table(current_table_pages_numbers):
                            all_tables.append(
                                TableAnnotation(
                                    pages_numbers=current_table_pages_numbers,
                                    pages=current_table_pages_text
                                )
                            )
                        current_table_pages_numbers = [page_num]
                        current_table_pages_text = [page]
                    else:
                        # No intervening text, so it's a continuation
                        current_table_pages_numbers.append(page_num)
                        current_table_pages_text.append(page)
                else:
                    if is_current_table(current_table_pages_numbers):
                        all_tables.append(
                            TableAnnotation(
                                pages_numbers=current_table_pages_numbers,
                                pages=current_table_pages_text
                            )
                        )
                    current_table_pages_numbers = [page_num]
                    current_table_pages_text = [page]
                last_table_page = page_num
        if is_current_table(current_table_pages_numbers):
            all_tables.append(
                TableAnnotation(
                    pages_numbers=current_table_pages_numbers,
                    pages=current_table_pages_text
                )
            )
    return all_tables


def extract_text_from_pdf(pdf_path):
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(pdf_path)
    text_by_page = {}

    # Extract text from each page
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
        text_by_page[page_num] = text

    return text_by_page

def get_pages_from_pdf(pdf_path):
    def remove_overlap_pages_with_table(cleaned_text_by_page, table: TableAnnotation):
        pages_text_with_table = ""
        for page_table in table.pages_numbers:
            pages_text_with_table = pages_text_with_table + " " + cleaned_text_by_page[page_table]
        text_with_summarized_pages = TableUtils.add_annotation_of_table(add_annotation_of_table_chain, {"input": pages_text_with_table})

        if text_with_summarized_pages and len(table.pages_numbers) > 0:
            cleaned_text_by_page[table.pages_numbers[0]] = text_with_summarized_pages
            for page_table in table.pages_numbers[1:]:
                cleaned_text_by_page.pop(page_table, None)

        return cleaned_text_by_page

    text_by_page = extract_text_from_pdf(pdf_path)
    pages_with_tables = extract_tables_from_pdf(pdf_path)
    cleaned_text_by_page = remove_header_and_footer_from_pages(text_by_page, pages_with_tables)
    for table in pages_with_tables:
        cleaned_text_by_page = remove_overlap_pages_with_table(cleaned_text_by_page, table)

    return concatenate_pages(cleaned_text_by_page)


def clean_first_page_of_legal_act(text):
    return re.split("(?=U\s*S\s*T\s*A\s*W\s*A)", text)[-1]


def remove_footer_from_page(text):
    split_point = list(re.finditer("\\n\s+\\n", text))
    if split_point:
        last_split = split_point[-1].end()
        text = text[:last_split].strip()
    return text


def remove_header_and_footer_from_pages(text_by_page, pages_with_tables):
    def flat_map_pages(pages):
        return [page for page_table in pages for page in page_table.pages_numbers]
    cleaned_text_by_page = {}
    for index, page_num in enumerate(text_by_page):
        cleaned_text_by_page[page_num] = remove_header_and_footer_from_single_page(text_by_page[page_num],
                                                                                   page_num == 0,
                                                                                   page_num not in flat_map_pages(pages_with_tables))

    return cleaned_text_by_page


def remove_header_and_footer_from_single_page(text_page, first_page=False, remove_footer=True):
    cleaned_text = re.sub("^©\s*Kancelaria\s+Sejmu\s*s\.\s+\d+/\d+\s*\\n\s*\\n\d{4}\s*[-]?\s*\d{2}\s*[-]?\s*\d{2}",
                          '', text_page)
    if first_page:
        cleaned_text = clean_first_page_of_legal_act(cleaned_text)
    if remove_footer:
        cleaned_text = remove_footer_from_page(cleaned_text)
    return cleaned_text


def clean_text(text: str) -> str:
    # Replace newlines, carriage returns, and multiple spaces with a single space
    return re.sub(r'[\s\n\r]+', ' ', text).strip()


def concatenate_pages(cleaned_text_by_page):
    full_text = ""
    for index, (page_num, text) in enumerate(cleaned_text_by_page.items()):
        full_text += text + " "
    return clean_text(full_text)


def get_annotations_of_changes(legal_act_text):
    annotation_of_changes_list = find_annotations_of_changes(legal_act_text)
    annotations_list = []
    for annotation_text in annotation_of_changes_list:
        response = annotations_of_changes_chain.call_chain({"input": annotation_text})
        if response:
            response.cleared_input = clean_text(response.cleared_input)
            annotations_list.append(response)

    return annotations_list


def find_annotations_of_changes(legal_act_text):
    regexes = [
        r'([a-zA-Z.\d\s]{,50}wejd.{,7}\s*w \s*życie\s*z\s*dn\.\s*\d{1,2}\.\d{1,2}\.\d{4}\s*r\.\s*[\sa-zA-Z.(\d]*\){1}\s*\.{1})'
    ]

    return [clean_text(match) for match in re.findall(regexes[0], legal_act_text)]


def remove_annotations_from_legal_act(legal_act_text, annotations):
    for annotation in annotations:
        legal_act_text = remove_annotation_from_legal_act(legal_act_text, annotation.cleared_input, 0)
    return legal_act_text


def remove_annotation_from_legal_act(legal_act_text: str, response_input: str, retry: int) -> str:
    pattern = re.compile(re.escape(response_input), re.IGNORECASE)
    match = pattern.search(legal_act_text)
    if retry > 3:
        print(f"Failed to remove annotation '{response_input}' from the text. Error!!!, retry limit exceeded")
        return legal_act_text
    if match:
        return clean_text(re.sub(re.escape(response_input), " ", legal_act_text, flags=re.IGNORECASE))
    else:
        print(f"Annotation '{response_input}' not found in the text. Error!!! , retrying..., retry: {retry + 1}")
        try:
            response_retry = get_annotation_retry_for_removing_chain.call_chain({"input": response_input})
            return remove_annotation_from_legal_act(legal_act_text, response_retry.cleared_input, retry + 1)
        except Exception as e:
            print(
                f"Failed to remove annotation '{response_input}' from the text. Error!!!, error during calling openai api")
            return legal_act_text


# text_by_page = extract_text_from_pdf("legal_acts/D19740141Lj.pdf")
# text_by_page = extract_text_from_pdf("legal_acts/D19830207Lj.pdf")
# text_by_page = extract_text_from_pdf("legal_acts/D19910031Lj.pdf")
# text_by_page = extract_text_from_pdf("legal_acts/D20110696Lj.pdf")
# text_by_page = extract_text_from_pdf("legal_acts/D20200321Lj.pdf")

cleaned_legal_act = get_pages_from_pdf("legal_acts/D20110696Lj.pdf")
# cleaned_legal_act = get_pages_from_pdf("legal_acts/D19830207Lj.pdf")

# l = find_annotations_of_changes(text_by_page)

annotation_of_changes_legal_act = get_annotations_of_changes(cleaned_legal_act)

legal_act_with_removed_annotation = remove_annotations_from_legal_act(cleaned_legal_act,
                                                                      annotation_of_changes_legal_act)
# cleaned_text_by_page = remove_header_footer(text_by_page)
# cleaned_legal_act = concatenate_pages(cleaned_text_by_page)
# f = 4
# cleaned_legal_act = remove_new_line(cleaned_legal_act)

r = 4

Annotation 'Nowe brzmienie ust. 1 w art. 34 wejdzie w zycie z dn. 1.07.2024 r. (Dz. U. z 2023 r. poz. 1938).' not found in the text. Error!!! , retrying..., retry: 1
Failed to remove annotation 'Nowe brzmienie ust. 1 w art. 34 wejdzie w zycie z dn. 1.07.2024 r. (Dz. U. z 2023 r. poz. 1938).' from the text. Error!!!, error during calling openai api
Annotation 'Nowe brzmienie ust. 2 w art. 34 wejdzie w zycie z dn. 1.07.2024 r. (Dz. U. z 2023 r. poz. 1938).' not found in the text. Error!!! , retrying..., retry: 1
Failed to remove annotation 'Nowe brzmienie ust. 2 w art. 34 wejdzie w zycie z dn. 1.07.2024 r. (Dz. U. z 2023 r. poz. 1938).' from the text. Error!!!, error during calling openai api


In [52]:
text = "   Hello, world!f   "
start_substring = "Hello"
end_substring = "world!"

# Strip leading and trailing spaces from text
stripped_text = text.strip()

if stripped_text.startswith(start_substring):
    print("Start substring found!")

if stripped_text.endswith(end_substring):
    print("End substring found!")


Start substring found!


In [35]:
www = extract_tables_from_pdf("legal_acts/D20110696Lj.pdf")

In [ ]:
t = '©Kancelaria Sejmu     s. 121/128

2024 -01-18
spożywczy specjalnego przeznaczenia żywieniowego, wyrób medyczny nie zostaje
umieszczony w  pierwszym wykazie, o  którym mowa w  art. 68.
6. Informacje, o  których m owa w  ust. 2, w zakresie określonym w  art. 26 pkt  1
lit. e–f dotyczą okresu od 1  maja 2010  r. do 30  kwietnia 2011  r.
7. Negocjacje w  sprawie ustalenia urzędowej ceny zbytu dotyczą proponowanej
w informacji przedstawionej przez wnioskodawcę wskazanego w  ust. 3 ceny zbytu
netto.
8. Urzędowa cena zbytu ustalona w  wyniku negocjacji nie może być wyższa niż
maksymalna cena detaliczna brutto leku, środka spożywczego specjalnego
przeznaczenia żywieniowego, wyrobu medycznego określona w  wykazie, o  którym
mowa w  art. 36 ust.  5, art.  37 ust.  2 i art. 38 ust.  6 ustawy, o  której mowa w  art. 637)
w brzmieniu obowiązującym przed dniem wejścia w  życie niniejszej ustawy,
pomniejszo na o  marżę hurtową w  wysokości 8,91% liczoną od ceny urzędowej
hurtowej oraz marżę detaliczną w  wysokości:

Cena hurtowa w  złotych
Marża detaliczna liczona od ceny
hurtowej
0–3,60 40%
       3,61–4,80 1,44 zł
4,81–6,50 30%
          6,51–9,75 1,95 zł
9,76–14,00  20%
            14,01–15,55  2,80 zł
15,56–30,00  18%
             30,01–33,75  5,40 zł
33,76–50,00  16%
             50,01–66,67  8,00 zł
66,68–100,00  12%
              powyżej 100,00  12,00  zł
9. Negocjacje przeprowadza Komisja. Do negocjacji stosuje się art.  18 ust.  3,
art. 19 ust.  1 i 2 pkt  2–7, art.  20 i art. 22 oraz przepisy wykonawcze wydane na
podstawie art.  23.
10. Minister właściwy do spraw zdrowia wydaje decyzję, o  której mowa
w art. 11 ust.  1, uwzględniając:
1) stanowisko Komisji,  '

In [40]:
from pdfplumber.page import Page


def create_annotation_of_table_for_llm(tables_pages: list):
    for table_annotation in tables_pages:
        for page in table_annotation.pages:
            page_table = page.extract_tables()

            ttt = page.find_tables()
            page_text = page.extract_text()
            for table_num, table in enumerate(page_table):
                k = str(table[0][0]).replace('\n', ' ')

                table_start_pos = page_text.find(str(table[0][0]).replace('\n', ' ')[:20])
                l = 4

            print(page_text)
    print(tables_pages)


create_annotation_of_table_for_llm(www)

©Kancelaria Sejmu s. 15/128
2) ilość leku recepturowego, którego dotyczy odpłatność ryczałtowa, oraz sposób
obliczania kosztu sporządzania leku recepturowego
– biorąc pod uwagę dostępność do leków, bezpieczeństwo ich stosowania oraz postać
farmaceutyczną.
Art. 7. 1. Ustala się urzędową marżę hurtową w wysokości 6% ceny zbytu netto
leku, środka spożywczego specjalnego przeznaczenia żywieniowego lub wyrobu
medycznego, nie niższą niż 50 groszy i nie wyższą niż:
1) 150 zł – w przypadku, o którym mowa w art. 6 ust. 1 pkt 1;
2) 2000 zł – w przypadkach, o których mowa w art. 6 ust. 1 pkt 2 i 3.
2. Przedsiębiorcy prowadzący obrót hurtowy w rozumieniu art. 72 ust. 3 ustawy
z dnia 6 września 2001 r. – Prawo farmaceutyczne są obowiązani stosować cenę zbytu
netto wraz z marżą, o której mowa w ust. 1, oraz właściwą stawkę podatku od towarów
i usług niezależnie od miejsca dostawy leku, środka spożywczego specjalnego
przeznaczenia żywieniowego lub wyrobu medycznego będącego przedmiotem tego
obrotu.
3

In [71]:
find_annotations_of_changes(cleaned_legal_act)

['Nowe brzmienie ust. 4 w art. 7 wejdzie w życie z dn. 1.01.2025 r. (Dz. U. z 2023 r. poz. 1938).',
 'cej iloczyn Nowe brzmienie ust. 1 w art. 34 wejdzie w życie z dn. 1.07.2024 r. (Dz. U. z 2023 r. poz. 1938).',
 'o jego wytworzenia. Dodane ust. 1a i 1b w art. 34 wejdą w życie z dn. 1.07.2024 r. (Dz. U. z 2023 r. poz. 1938).',
 'Nowe brzmienie ust. 2 w art. 34 wejdzie w życie z dn. 1.07.2024 r. (Dz. U. z 2023 r. poz. 1938).']

In [82]:
cleaned_legal_act

'U S T AWA z dnia 12 maja 2011 r. o refundacji leków, środków spożywczych specjal nego przeznaczenia żywieniowego oraz wyrobów medycznych1) Rozdział 1 Przepisy ogólne Art. 1. Ustawa określa: 1) zasady, warunki i tryb podejmowania decyzji administracyjnej o objęciu refundacją leku, środka spożywczego specjalnego przeznaczenia żywi eniowego, wyrobu medycznego i uchylenia tej decyzji; 1a) zakres tajemnicy refundacyjnej; 2) zasady finansowania leku, środka spożywczego specjalnego przeznaczenia żywieniowego, wyrobu medycznego objętych decyzją, o której mowa w pkt 1; 3) kryteria tworzenia poziomów odpłatności i grup limitowych leków, środków spożywczych specjalnego przeznaczenia żywieniowego, wyrobów medycznych objętych decyzją, o której mowa w pkt 1; 4) zasady i tryb oraz kryteria ustalania cen zbytu netto na leki, środki spożywc ze specjalnego przeznaczenia żywieniowego, wyroby medyczne, a także wysokość urzędowych marż hurtowych i urzędowych marż detalicznych; 5) zasady ustalania cen lekó

In [93]:
full_text = legal_act_with_removed_annotation


In [94]:
full_text

'U S T AWA z dnia 12 maja 2011 r. o refundacji leków, środków spożywczych specjal nego przeznaczenia żywieniowego oraz wyrobów medycznych1) Rozdział 1 Przepisy ogólne Art. 1. Ustawa określa: 1) zasady, warunki i tryb podejmowania decyzji administracyjnej o objęciu refundacją leku, środka spożywczego specjalnego przeznaczenia żywi eniowego, wyrobu medycznego i uchylenia tej decyzji; 1a) zakres tajemnicy refundacyjnej; 2) zasady finansowania leku, środka spożywczego specjalnego przeznaczenia żywieniowego, wyrobu medycznego objętych decyzją, o której mowa w pkt 1; 3) kryteria tworzenia poziomów odpłatności i grup limitowych leków, środków spożywczych specjalnego przeznaczenia żywieniowego, wyrobów medycznych objętych decyzją, o której mowa w pkt 1; 4) zasady i tryb oraz kryteria ustalania cen zbytu netto na leki, środki spożywc ze specjalnego przeznaczenia żywieniowego, wyroby medyczne, a także wysokość urzędowych marż hurtowych i urzędowych marż detalicznych; 5) zasady ustalania cen lekó

In [95]:
class PassageSplitOpenAIError(Exception):
    pass


class PassageSplitError(Exception):
    pass


class GetTextBetweenBracketsError(Exception):
    pass


class EraseNumberError(Exception):
    pass

In [103]:
from abc import abstractmethod, ABC


class TextChangesPiecesSplitter(ABC):
    UPCOMING_CHANGE = "UPCOMING_CHANGE"
    BEFORE_UPCOMING_CHANGE = "BEFORE_UPCOMING_CHANGE"
    NOT_MATCH_CHANGE = "NON_MATCH"
    BEFORE_UPCOMING_CHANGE_PATTERN = r'^\[([^\[\]]*?)\]$'
    UPCOMING_CHANGE_PATTERN = r'^<([^\[\]]*?)>$'

    def __init__(self):
        pass

    def get_pattern_for_text_with_brackets(self, action_type):
        return self.BEFORE_UPCOMING_CHANGE_PATTERN if action_type == self.BEFORE_UPCOMING_CHANGE else self.UPCOMING_CHANGE_PATTERN

    def _find_changes_text(self, legal_text: str, pattern_str: str, action_type: str):
        pattern = re.compile(pattern_str)
        pattern_between_brackets = self.get_pattern_for_text_with_brackets(action_type)

        changes = []
        for match in pattern.finditer(legal_text):
            is_text_between_brackets = re.match(pattern_between_brackets, match.group())
            if is_text_between_brackets:
                changes.append((match.start(), match.end(), is_text_between_brackets.group(1), action_type))
            else:
                raise GetTextBetweenBracketsError(f"Error during getting text between brackets in '{match.group()}'")
        return changes

    def _organise_text(self, text, before_upcoming_changes, after_upcoming_changes):
        upcoming_changes = before_upcoming_changes + after_upcoming_changes
        non_matches_of_not_upcoming_changes = self._find_non_matches(upcoming_changes, text)
        all_pieces = self.sort_pieces_of_art_text(upcoming_changes + non_matches_of_not_upcoming_changes)
        if self._are_pieces_overlap(all_pieces):
            raise PassageSplitError("Upcoming changes are overlapping")
        return [(left_i, right_i, text.strip(), action_type) for left_i, right_i, text, action_type in all_pieces]

    def sort_pieces_of_art_text(self, pieces):
        pieces.sort(key=lambda x: (x[0], x[1]))
        return pieces

    def _are_pieces_overlap(self, pieces):
        for i in range(len(pieces) - 1):
            if pieces[i][1] > pieces[i + 1][0]:
                return True
        return False

    def _find_non_matches(self, upcoming_changes, art_text: str):
        def add_non_match(prev_end, next_start):

            if prev_end < next_start and art_text[prev_end:next_start].strip() != "":
                return [(prev_end, next_start, art_text[prev_end:next_start], self.NOT_MATCH_CHANGE)]
            return []

        sorted_upcoming_changes = self.sort_pieces_of_art_text(upcoming_changes)
        non_matches = []
        last_end = 0
        for start, end, text, action_type in sorted_upcoming_changes:
            non_matches += add_non_match(last_end, start)
            last_end = end
        non_matches += add_non_match(last_end, len(art_text))
        return non_matches

    def split(self, text):
        part_of_texts = self._organise_text(text,
                                            self._find_changes_text(text, self.before_upcoming_change_pattern(),
                                                                    self.BEFORE_UPCOMING_CHANGE),
                                            self._find_changes_text(text, self.upcoming_change_pattern(),
                                                                    self.UPCOMING_CHANGE)
                                            )
        return [
            {'text': self.process_text(split.strip()), 'action_type': action_type}
            for start_index, end_index, part_text, action_type in part_of_texts
            for split in self.split_function(part_text)
        ]

    def get_identificator_text(self, split):
        if self.can_erase_number(split) is None:
            raise EraseNumberError("Error during getting identificator text")
        return re.findall(self.can_erase_number_pattern(), split['text'])[0].strip()

    def erase_identificator_from_text(self, split):
        if self.can_erase_number(split) is None:
            raise EraseNumberError("Error during getting identificator text")
        return re.sub(self.can_erase_number_pattern(), "", split['text']).strip()

    def can_erase_number(self, split):
        return re.match(self.can_erase_number_pattern(), split['text'])

    @abstractmethod
    def can_erase_number_pattern(self):
        pass

    @abstractmethod
    def before_upcoming_change_pattern(self):
        pass  # <....>

    @abstractmethod
    def upcoming_change_pattern(self):
        pass  # [....]

    @abstractmethod
    def split_function(self, text):
        pass

    @abstractmethod
    def process_text(self, text):
        pass


class ArticleSplitter(TextChangesPiecesSplitter):
    ERASE_FORMAT_legal_acts_structure_words_from_art_pattern = r'\s*(?=R\s*o\s*z\s*d\s*z\s*i\s*a\s*[łl]|D\s*Z\s*I\s*A\s*[ŁL]|O\s*d\s*d\s*z\s*i\s*a\s*[łl])'

    def __init__(self):
        super().__init__()

    def before_upcoming_change_pattern(self):
        return r"\[\s*Art\.\s*\d+[a-zA-Z]*\.[^\]\[\>]*\]"  # [Art. 1.]

    def upcoming_change_pattern(self):
        return r"<\s*Art\.\s*\d+[a-zA-Z]*\.[^>\]\[]*>"  # <Art. 1.>

    def split_function(self, text):
        return re.findall(r'(Art\. \d+[a-zA-Z]?[\s\S]*?)(?=Art\. \d+[a-zA-Z]?|$)', text)

    def can_erase_number_pattern(self):
        return r'Art\.\s*(\d*\w*(?:–\d*\w*)?).\s*'

    def process_text(self, text):
        return self.erase_redundant_act_text(text)

    def erase_redundant_act_text(self, act_text):
        return re.split(r'\s*(?=R\s*o\s*z\s*d\s*z\s*i\s*a\s*[łl]|D\s*Z\s*I\s*A\s*[ŁL]|O\s*d\s*d\s*z\s*i\s*a\s*[łl])',
                        act_text)[0]


class PassageSplitter(TextChangesPiecesSplitter):
    def __init__(self):
        super().__init__()

    def before_upcoming_change_pattern(self):
        return r"\[\s*\d+[a-z]*\.[^\]\<\>]*\]"  # [1.]

    def upcoming_change_pattern(self):
        return r"<\s*\d+[a-z]*\.[^>\]\[]*>"  # <1.>

    def split_function(self, text):
        return re.split(r'(?=[\sa-zA-Z[<]\d{1,}[a-z]*\.\s[\D(])', text)

    def can_erase_number_pattern(self):
        return r"^\s*(\d+[a-z]*)\s*\.\s*(?=\w|\[|\()"

    def process_text(self, text):
        return self.remove_redudant_section_sign(
            self.standardize_section_reference_keyword(text)
        )

    def map_splits_from_api_retry(self, passage_splits: List):
        def process_split(passage_split_text):
            if re.match(self.get_pattern_for_text_with_brackets(self.BEFORE_UPCOMING_CHANGE), passage_split_text):
                return {
                    "text": re.match(self.get_pattern_for_text_with_brackets(self.BEFORE_UPCOMING_CHANGE),
                                     passage_split_text).group(1).strip(),
                    "action_type": self.BEFORE_UPCOMING_CHANGE
                }
            elif re.match(self.get_pattern_for_text_with_brackets(self.UPCOMING_CHANGE), passage_split_text):
                return {
                    "text": re.match(self.get_pattern_for_text_with_brackets(self.UPCOMING_CHANGE),
                                     passage_split_text).group(1).strip(),
                    "action_type": self.UPCOMING_CHANGE
                }
            else:
                return {
                    "text": passage_split_text.strip(),
                    "action_type": self.NOT_MATCH_CHANGE
                }

        return list(map(process_split, passage_splits))

    def remove_redudant_section_sign(self, passage_text):
        return passage_text.strip().rstrip("§") if passage_text.rstrip().endswith('§') \
            else passage_text.strip()

    def standardize_section_reference_keyword(self, passage_text):
        return passage_text.replace('§', 'ust.')


class SubpointSplitter(TextChangesPiecesSplitter):
    # do the same correct splitting usinng openai because some error could occurs during splitting
    def __init__(self):
        super().__init__()

    def before_upcoming_change_pattern(self):
        return r"\[\s*\d+[a-z]*\)[^\]\<\>]*\]"  # [1)]

    def upcoming_change_pattern(self):
        return r"<\s*\d+[a-z]*\)[^>\]\[]*>"  # <1)>

    def split_function(self, text):
        return re.split(r'(?=(?<=[;):,])\s+(?=\d+[a-z]*\)\s*[\w|\[|\(]))', text)

    def can_erase_number_pattern(self):
        return r"^\s*(\d+[a-z]*)\s*\)\s*(?=\w|\[|\()"

    def process_text(self, text):
        return text


class SubSubpointSplitter(TextChangesPiecesSplitter):
    def __init__(self):
        super().__init__()

    def before_upcoming_change_pattern(self):
        return r"\[\s*[a-z]+\)[^\]\<\>]*\]"  # [a)]

    def upcoming_change_pattern(self):
        return r"<\s*[a-z]+\)[^>\]\[]*>"  # <a)>

    def split_function(self, text):
        return re.split(r'(?=(?<=[;):,])\s*(?=[a-z]+\s*\)))', text)

    def can_erase_number_pattern(self):
        return r"^\s*([a-z]*)\s*\)\s*"

    def process_text(self, text):
        return text



In [109]:
import re

class FragmentWithFunction:
    def __init__(self, function_definition):
        self.function_definition = function_definition

class DefinitionTableFunction(BaseModel):
    function_name: str  # where is name of function which handle table
    function_definition: str  # where is definition of function


class ArticleTemp:
    def __init__(self, art_number, art_text, passages, subpoints, action_type):
        self.art_number = art_number
        self.art_text = art_text
        self.passages = passages
        self.subpoints = subpoints
        self.action_type = action_type


class PassageTemp(FragmentWithFunction):
    def __init__(self, passage_number, passage_text, subpoints, action_type, function_definition=None):
        super().__init__(function_definition)
        self.passage_number = passage_number
        self.passage_text = passage_text
        self.subpoints = subpoints
        self.action_type = action_type


class SubpointTemp(FragmentWithFunction):
    def __init__(self, subpoint_number, subpoint_text, subSubpointsRefs, action_type, function_definition=None):
        super().__init__(function_definition)
        self.subpoint_number = subpoint_number
        self.subpoint_text = subpoint_text
        self.subSubpointsRefs = subSubpointsRefs
        self.action_type = action_type


def is_up_to_date_piece(text):
    normalized_text = text.replace(" ", "")
    # Check if the normalized text is exactly "(uchylony)", "(pominięte)", or "(pominięty)"
    return normalized_text not in ["(uchylony)", "(uchylona)", "(pominięte)", "(pominięty)"]


def passage_splitting_call(text_for_split_art) -> PassageSplits:
    passage_split_chain = passage_split_prompt | openAIModel.with_structured_output(PassageSplits)
    error_messages = []

    try:
        response = passage_split_chain.invoke({"input": text_for_split_art})

        return response
    except (openai.error.InvalidRequestError, openai.error.AuthenticationError) as e:
        # Handle invalid request error or authentication error
        error_messages.append(f"Invalid request or authentication error: {e}")
    except (openai.error.APIConnectionError, openai.error.RateLimitError) as e:
        # Handle API connection error or rate limit error
        error_messages.append(f"API connection or rate limit error: {e}")
    except Exception as e:
        # Handle any other unexpected error
        error_messages.append(f"An unexpected error occurred: {e}")

    if error_messages:
        print(" | ".join(error_messages))
    raise PassageSplitOpenAIError("Error during calling openai api for passage splitting")



def extract_parts(passage):
    # Use regex to extract numeric and alphabetic parts
    match = re.match(r"(\d+)([a-z]?)", passage)
    if match:
        num_part = int(match.group(1))  # Convert numeric part to an integer
        alpha_part = match.group(2)  # Alphabetic part remains a string
        return (num_part, alpha_part)

    return (float('inf'), '')  # Fallback if the regex fails


def is_ascending(passage_numbers):
    # Extract parts for sorting
    extracted_parts = [extract_parts(passage) for passage in passage_numbers]
    # Check if the list is sorted
    return extracted_parts == sorted(extracted_parts)

def is_blank_passage(passage: PassageTemp):
    return passage.passage_text.strip() == ""


def remove_redudant_section_sign(passage_list):
    return [item.strip().rstrip("§") if item.rstrip().endswith('§') else item.strip() for item in passage_list]


def standardize_section_reference_keyword(passage_list):
    return [item.replace('§', 'ust.') for item in passage_list]


class CustomMarkdownSplitter:
    def __init__(self):
        self.article_splitter = ArticleSplitter()
        self.passage_splitter = PassageSplitter()
        self.subpoint_splitter = SubpointSplitter()
        self.subsubpoint_splitter = SubSubpointSplitter()

        self.erase_format_legal_acts_structure_words_from_art_pattern = r'\s*(?=R\s*o\s*z\s*d\s*z\s*i\s*a\s*[łl]|D\s*Z\s*I\s*A\s*[ŁL]|O\s*d\s*d\s*z\s*i\s*a\s*[łl])'
        self.table_start_pattern = r'\/\s*t\s*a\s*b\s*l\s*e\s*_\s*s\s*t\s*a\s*r\s*t\s*\/'

        self.create_function_from_table_chain = LLMChain(
            model=model4o,
            output_schema=DefinitionTableFunction,
            system_massage=("system", '''
            Na podstawie poniższego tekstu, który zawiera tabelę, stwórz funkcję w Pythonie, która będzie w stanie obsłużyć dane z tej tabeli. Zdefiniuj odpowiednią nazwę funkcji oraz jej argumenty, biorąc pod uwagę specyfikacje zawarte w tabeli.
            1. Upewnij się, że funkcja poprawnie przetwarza wszystkie kolumny i wiersze tabeli oraz spełnia wymagania zawarte w tekście. Funkcja powinna być elastyczna i działać prawidłowo niezależnie od wartości podanych w tabeli.
            2. Dodaj zwięzły i jasny komentarz w formie docstringa wewnątrz funkcji, zawierający:
                - Opis działania funkcji.
                - Informacje o przyjmowanych argumentach, w tym ich typach i jednostkach.
                - Informacje o zwracanej wartości.
                - Dodatkowe uwagi, które mogą być istotne w kontekście interpretacji danych z tabeli.
            3. Używaj wyłącznie języka polskiego przy definiowaniu funkcji, argumentów i docstringów.
            4. Zwróć funkcję w poprawnym formacie kodu Python, tak aby można było ją natychmiast wykorzystać w programie.

        '''),
            user_massage=("user", "{input}")
        )
        self.erase_table_and_summarize_table_chain = LLMChain(
            model=model4o,
            output_schema=LegalActWithErasedTableWithAnnotation,
            system_massage=("system", '''
            1. Na podstawie wcześniej znalezionej tabeli w poprzedniej odpowiedzi
            2. usuń tę tabelę z tekstu wraz z wszystkimi!!! informacjami ją opisującymi jak zmienne odniesienia, wartości, itp.
            3  nie usuwaj "/table_start/" w tekście !!!!
            4. Zachowaj numerację, taką jak 1. 1) a).
            5. Zwróć wynik jako ciąg tekstowy bez dodatkowych formatowań, takich jak indeksy czy nowe linie.
        '''),
            system_massage_retry=("system", '''
            1. Na podstawie wcześniej znalezionej tabeli w poprzedniej odpowiedzi
            2. usuń tę tabelę z tekstu wraz z wszystkimi!!! informacjami ją opisującymi jak zmienne odniesienia, wartości, itp.
            3  nie usuwaj "/table_start/" w tekście !!!!
            4. Zachowaj numerację, taką jak 1. 1) a).
            5. Zwróć wynik jako ciąg tekstowy bez dodatkowych formatowań, takich jak indeksy czy nowe linie.

            Poprzednia wiadomość była nie poprawna spróbuj jeszcze raz.
        '''),
            user_massage=("human", "{input}"),
            few_shot_examples=table_erasing_examples
        )

        # self.text_changes_splitter = TextChangesPiecesSplitter()

        self.kupa = 0

    def split_text(self, text):
        articles_splits = self.article_splitter.split(text)

        articles = []

        for split in articles_splits:

            art_number = self.article_splitter.get_identificator_text(split)
            cleaned_art_text = self.article_splitter.erase_identificator_from_text(split)
            if is_up_to_date_piece(cleaned_art_text):
                articles.append(self.split_passage(
                    art_number=art_number,
                    text_for_split_art=cleaned_art_text,
                    action_type_art=split["action_type"]
                )
                )

        return articles

    def split_passage(self, art_number, text_for_split_art, action_type_art):
        passage_splits = self.passage_splitter.split(text=text_for_split_art)
        try:
            article = self.process_passages(art_number=art_number,
                                            action_type_art=action_type_art,
                                            passage_splits=passage_splits)
        except PassageSplitError as e:
            try:
                passage_splits_text = passage_splitting_call(text_for_split_art=text_for_split_art).passage_splits
                passage_splits_retried = self.passage_splitter.map_splits_from_api_retry(passage_splits_text)
                article = self.process_passages(art_number=art_number,
                                                action_type_art=action_type_art,
                                                passage_splits=passage_splits_retried,
                                                is_from_openai_call=True)
            except PassageSplitOpenAIError as e:
                raise PassageSplitError("Error during calling openai api for passage splitting, it shouldn't happen")
            except PassageSplitError as e:
                raise PassageSplitError("Error during splitting passages after calling openai api, it shouldn't happen")

        return article

    def is_there_table_in_passage_split(self, passage_split):
        return re.search(self.table_start_pattern, passage_split['text'])


    def process_passages(self, art_number, action_type_art, passage_splits,
                         is_from_openai_call=False):

        article = ArticleTemp(
            art_number=art_number,
            art_text=None,
            passages=[],
            subpoints=[],
            action_type=action_type_art
        )

        if art_number == "38c":
            r = 4
        for index_split, passage_split in enumerate(passage_splits):
            table_function_passage = None
            if not isinstance(passage_split, dict):
                r = 4

            section_match = self.passage_splitter.can_erase_number(passage_split)
            if index_split == 0 and not section_match:
                art_text, subpoints = self.split_subpoint(
                    passage_number=None,
                    passage_for_further_split=passage_split['text'],
                    passage_action_type=passage_split['action_type'],
                    table_function_passage=table_function_passage
                )
                article.art_text = art_text
                article.subpoints.extend(subpoints)
            elif section_match:
                section_number = self.passage_splitter.get_identificator_text(passage_split)
                cleaned_passage_text = self.passage_splitter.erase_identificator_from_text(passage_split)
                if self.is_there_table_in_passage_split(passage_split):
                    table_function_passage = TableUtils.create_function_from_table(self.create_function_from_table_chain, {"input": cleaned_passage_text})
                    cleaned_passage_text = TableUtils.summarize_table_and_erase_it_from_passage(self.erase_table_and_summarize_table_chain, {"input": cleaned_passage_text})
                if cleaned_passage_text == '' and is_from_openai_call:
                    print("Error from subpoints!!! \n")

                if is_up_to_date_piece(cleaned_passage_text):
                    passage = self.split_subpoint(
                        passage_number=section_number,
                        passage_for_further_split=cleaned_passage_text,
                        passage_action_type=passage_split['action_type'],
                        table_function_passage=table_function_passage
                    )
                    article.passages.append(passage)


            else:
                # is_error = True
                k = 4
                if is_from_openai_call:
                    print("ERROR from passage from second !!!!!!! \n")

        if any([is_blank_passage(passage) for passage in article.passages]):
            raise PassageSplitError("Some passages are blank")
        if not is_ascending([passage.passage_number for passage in article.passages]):
            raise PassageSplitError("Passage numbers are not in ascending order")
        return article

    def split_subpoint(self, passage_number, passage_for_further_split, passage_action_type, table_function_passage):
        sub_points_splits = self.subpoint_splitter.split(text=passage_for_further_split)
        is_nested_in_passage = passage_number is not None

        subpoints = [] if not is_nested_in_passage else None
        passage = PassageTemp(
            passage_number=passage_number,
            passage_text=None,
            subpoints=[],
            action_type=passage_action_type,
            function_definition=table_function_passage
        ) if is_nested_in_passage else None

        art_text = None
        for sub_points_index, sub_point_split in enumerate(sub_points_splits):
            sub_point_match = self.subpoint_splitter.can_erase_number(sub_point_split)
            if sub_points_index == 0 and not sub_point_match and is_nested_in_passage:
                passage.passage_text = sub_point_split['text']
            elif sub_points_index == 0 and not sub_point_match:
                art_text = sub_point_split['text']
            elif sub_point_match:
                sub_point_number = self.subpoint_splitter.get_identificator_text(sub_point_split)
                cleaned_subpoint_text = self.subpoint_splitter.erase_identificator_from_text(sub_point_split)
                if cleaned_subpoint_text == '' and is_nested_in_passage:
                    print("Error from subpoints!!! something is wrong fdfndjn\n")
                if is_up_to_date_piece(cleaned_subpoint_text):
                    subpoint = self.create_subpoint(sub_point_number, cleaned_subpoint_text,
                                                    sub_point_split['action_type'])
                    if is_nested_in_passage:
                        passage.subpoints.append(subpoint)
                    else:
                        subpoints.append(subpoint)
            else:
                print("Error from subpoints  gfbhgfbhjf!!! \n")

        if is_nested_in_passage and passage.passage_text == None:
            RRRR = 4

        if not is_nested_in_passage and not is_ascending([subpoint.subpoint_number for subpoint in subpoints]):
            print("Error from subpoints!!!  is really bad error\n")
        if is_nested_in_passage and not is_ascending([subpoint.subpoint_number for subpoint in passage.subpoints]):
            print("Error from subpoints!!!  is really bad error\n")

        return passage if is_nested_in_passage else (art_text, subpoints)

    def create_subpoint(self, sub_point_number, subpoint_text, sub_point_action_type):
        sub_sub_points_splits = self.subsubpoint_splitter.split(subpoint_text)
        sub_sub_points = []
        final_text = ""
        for sub_sub_split in sub_sub_points_splits:
            sub_sub_match = self.subsubpoint_splitter.can_erase_number(sub_sub_split)
            if sub_sub_match:
                sub_sub_sign = self.subsubpoint_splitter.get_identificator_text(sub_sub_split)
                sub_sub_cleaned_text = self.subsubpoint_splitter.erase_identificator_from_text(sub_sub_split)
                if is_up_to_date_piece(sub_sub_cleaned_text):
                    final_text = final_text + " " + sub_sub_split['text']
                    sub_sub_points.append(sub_sub_sign)
            else:
                final_text = final_text + " " + sub_sub_split['text']
        if final_text == "":
            print("Error from subpoints!!!  is really bad error\n")
        return SubpointTemp(sub_point_number, final_text.strip(), sub_sub_points, sub_point_action_type)


# Initialize the custom splitter
custom_splitter = CustomMarkdownSplitter()

arts = custom_splitter.split_text(full_text)
#
# l = custom_splitter.split_legal_act_text(ttt)

In [104]:
r = 4

In [36]:
text_sraka = """
 [2. Przez niedotrzymanie zobowiązania dotyczącego ciągłości dostaw, o którym mowa w ust. 1, rozumie się brak obrotu produktem objętym re fundacją stwierdzony na podstawie raportów, o których mowa w art. 78 ust. 1 pkt 6 ustawy z dnia 6 września 2001 r. – Prawo farmaceutyczne lub informacji, o których mowa w art. 190 ust. 2 ustawy o świadczeniach. ] <2. Przez niedotrzymanie zobowiązania dotyczącego ciągłości dostaw, o którym mowa w ust. 1, rozumie się brak obrotu hurtowego w rozumieniu art. 72 ust. 3 ustawy z dnia 6 września 2001 r. – Prawo farm aceutyczne, produktem objętym refundacją, polegający na dostarczeniu produktu do świadczeniodawcy lub hurtowni farmaceutycznych, zlokalizowanych na terytorium Rzeczypospolitej Polskiej, w okresie danego kwartału kalendarzowego w ilościach mniejszych niż wy nikających ze zobowiązania do miesięcznych dostaw tego kwartału, określonych w decyzji administracyjnej o objęciu refundacją, lub zobowiązania, o którym mowa w art. 25 pkt 4, chyba że niewykonywanie tego zobowiązania jest następstwem działania siły wyższej , a w przypadku produktu leczniczego terapii zaawansowanej – niedotrzymanie zobowiązania do zapewnienia gotowości technologicznej do jego wytworzenia. Powyższego obowiązku nie stosuje się do leków będących przedmiotem przetargów centralnych prowadzonych pr zez Prezesa Funduszu w przypadku wyłonienia w tym przetargu dostawcy leków dla całej populacji chorych w danym wskazaniu refundacyjnym na okres co najmniej 12 miesięcy oraz do leków dostarczanych sezonowo. > 3. Przez niedotrzymanie zobowiązania dotyczącego wielkości rocznych dostaw, o którym mowa w ust. 1, rozumie się niewprowadzenie w ciągu roku kalendarzowego do obrotu zadeklarowanej we wniosku o objęcie refundacją ilości leku, środka spożywczego specjalnego przeznaczenia żywieniowego, wyrobu medycznego. 3a. Wnioskodawca, w przypadku leków, środków spożywczych specjalnego przeznaczenia żywieniowego lub wyrobów medycznych objętych refundacją w kategorii dostępności refundacyjnej, o której mowa w art. 6 ust. 1 pkt 1, zagrożonych brakiem dostępności na teryto rium Rzeczypospolitej Polskiej, obowiązany jest dostarczać te produkty w równej części do co najmniej 10 przedsiębiorców prowadzących hurtownie farmaceutyczne, których zakres działalności, o którym mowa w art. 76 ust. 1 pkt 8 ustawy z dnia 6 września 2001 r. – Prawo farmaceutyczne, określony w załączniku do zezwolenia na prowadzenie hurtowni farmaceutycznej, nie zawiera ograniczeń asortymentu, na terenie Rzeczypospolitej Polskiej, z największym udziałem obrotu z aptekami ogólnodostępnymi pod względem liczby transakcji, w ilości koniecznej do zabezpieczenia pacjentów, o którym mowa w art. 36z ust. 1 tej ustawy. 3b. Wykaz hurtowni, o których mowa w ust. 3a, minister właściwy do spraw zdrowia ustala corocznie do dnia 30 kwietnia danego roku w oparciu o dane dotyczące obrotu refundowanymi lekami, środkami spożywczymi specjalnego przeznaczenia żywieniowego lub wyrobami medycznymi między hurtowniami farmaceutycznymi a aptekami ogólnodostępnymi za poprzedni rok kalendarzowy, zawarte w Zintegrowanym Systemie Monit orowania Obrotu Produktami Leczniczymi, o którym mowa w art. 72a ust. 1 ustawy z dnia 6 września 2001 r. – Prawo farmaceutyczne. 3c. Minister właściwy do spraw zdrowia publikuje w Biuletynie Informacji Publicznej wykaz, o którym mowa w ust. 3b. 3d. Wykaz l eków, środków spożywczych specjalnego przeznaczenia żywieniowego lub wyrobów medycznych, o których mowa w ust. 3a, minister właściwy do spraw zdrowia ustala w drodze obwieszczenia i ogłasza w Dzienniku Urzędowym ministra właściwego do spraw zdrowia, kieruj ąc się potrzebą zapewnienia możliwie równomiernego zabezpieczenia pacjentów. 4. Kwotę, o której mowa w ust. 1, oblicza Fundusz na podstawie danych, o których mowa w art. 102 ust. 5 pkt 31 ustawy o świadczeniach. 5. (uchylony) 6. Kwotę, o której mowa w ust. 1, ustala w drodze noty
"""

In [46]:
arts

In [33]:
import tiktoken

# Define the passage
passage = """
Ustala się kategorię dostępności refundacyjnej: 1) lek, środek spożywczy specjalnego przeznaczenia żywieniowego, wyrób medyczny dostępny w aptece na receptę: a) w cał ym zakresie zarejestrowanych wskazań i przeznaczeń, b) we wskazaniu określonym stanem klinicznym; 2) lek, środek spożywczy specjalnego przeznaczenia żywieniowego stosowany w ramach programu lekowego; 3) lek stosowany w ramach chemioterapii: a) w całym zakresie zarejestrowanych wskazań i przeznaczeń, b) we wskazaniu określonym stanem klinicznym; 4) lek, środek spożywczy specjalnego przeznaczenia żywieniowego , wyrób medyczny stosowany w ramach udzielania świadczeń gwarantowanych, innych niż wymienione w pkt 1–3. 2. Lek, środek spożywczy specjalnego przeznaczenia żywieniowego, wyrób medyczny, dla którego wydana została decyzja administracyjna o objęciu refundacją w zakresie nadanej kategorii dostępności refundacyjnej, o której mowa w ust. 1 pkt 1, jest wyda wany świadczeniobiorcy: 1) bezpłatnie, 2) za odpłatnością ryczałtową, 3) za odpłatnością w wysokości 30% albo 50% ich limitu finansowania – do wysokości limitu finansowania i za dopłatą w wysokości różnicy między ceną detaliczną a wysokością limitu finan sowania. 2a. Kwota odpłatności przez pacjenta, o której mowa w ust. 2, ulega obniżeniu o: 1) 10% – w przypadku gdy lek jest wytwarzany na terytorium Rzeczypospolitej Polskiej albo gdy do jego wytworzenia wykorzystano substancję czynną wytworzoną na terytor ium Rzeczypospolitej Polskiej, 2) 15% – w przypadku gdy lek jest wytwarzany na terytorium Rzeczypospolitej Polskiej z wykorzystaniem substancji czynnej wytwarzanej na terytorium Rzeczypospolitej Polskiej – przy czym koszt tej obniżki jest finansowany ze śr odków Fundusz
... [truncated for brevity] ...
"""

# Encoding the passage
encoding = tiktoken.encoding_for_model("gpt-4o")
num_tokens = len(encoding.encode(passage))

print("Number of tokens:", num_tokens)


Number of tokens: 562


In [32]:
import re


def extract_parts(passage):
    # Use regex to extract numeric and alphabetic parts
    match = re.match(r"(\d+)([a-z]?)", passage)
    if match:
        num_part = int(match.group(1))  # Convert numeric part to an integer
        alpha_part = match.group(2)  # Alphabetic part remains a string
        return (num_part, alpha_part)

    return (float('inf'), '')  # Fallback if the regex fails


def is_ascending(passage_numbers):
    # Extract parts for sorting
    extracted_parts = [extract_parts(passage) for passage in passage_numbers]
    # Check if the list is sorted
    return extracted_parts == sorted(extracted_parts)


# Example usage

passage_numbers = ['14', '14a']
print(is_ascending(passage_numbers))  # Output: True

passage_numbers = ['1', '2', '3', '4', '4b', '4a', '5']
print(is_ascending(passage_numbers))  # Output: True

# Test with unsorted list
unsorted_passages = ['1', '2', '4', '3', '4a', '4b', '5']
print(is_ascending(unsorted_passages))  # Output: False

True
False
False


In [26]:
kk = 'Art. 185. § 1. Społeczną kontrolę przestrzegania prawa pracy, w  tym przepisów \ni zasad bezpieczeństwa i  higieny pracy, sprawuje społeczna inspekcja pracy.  \n§ 2. Organizację, zadania i  uprawnienia społecznej inspekcji pracy oraz zasady \njej współdziałania z  Państwową Inspekcją Pracy i  innymi państwowymi organami \nnadzoru i  kontroli określają odrębne przepisy.  \nRozdział III  \n(zawierający art.  19–21 – uchylony)  \nDZIAŁ DRUGI  \nStosunek pracy  \nRozdział I  \nPrzepisy ogólne  \n'

erase_format_legal_acts_structure_words_from_art_pattern = r'\s*(?=R\s*o\s*z\s*d\s*z\s*i\s*a\s*[łl]|D\s*Z\s*I\s*A\s*[ŁL]|O\s*d\s*d\s*z\s*i\s*a\s*[łl])'
l = re.split(erase_format_legal_acts_structure_words_from_art_pattern, kk)

print(l)
print(len(l))
print(l[0])
print("-------")
print(l[1])
print(l[2])

['Art. 185. § 1. Społeczną kontrolę przestrzegania prawa pracy, w  tym przepisów \ni zasad bezpieczeństwa i  higieny pracy, sprawuje społeczna inspekcja pracy.  \n§ 2. Organizację, zadania i  uprawnienia społecznej inspekcji pracy oraz zasady \njej współdziałania z  Państwową Inspekcją Pracy i  innymi państwowymi organami \nnadzoru i  kontroli określają odrębne przepisy.', '', 'Rozdział III  \n(zawierający art.  19–21 – uchylony)', '', 'DZIAŁ DRUGI  \nStosunek pracy', '', 'Rozdział I  \nPrzepisy ogólne  \n']
7
Art. 185. § 1. Społeczną kontrolę przestrzegania prawa pracy, w  tym przepisów 
i zasad bezpieczeństwa i  higieny pracy, sprawuje społeczna inspekcja pracy.  
§ 2. Organizację, zadania i  uprawnienia społecznej inspekcji pracy oraz zasady 
jej współdziałania z  Państwową Inspekcją Pracy i  innymi państwowymi organami 
nadzoru i  kontroli określają odrębne przepisy.
-------

Rozdział III  
(zawierający art.  19–21 – uchylony)


In [13]:
arts

In [23]:
import re

text = "Art. 12. 1. Dla celów podatku  dochodowego od osób fizycznych dodatkowe \nroczne świadczenie pieniężne stanowi świadczenie, o  którym mowa w  art. 12 ust. 7 \nustawy z  dnia 26  lipca 1991  r. o podatku dochodowym od osób fizycznych (Dz. U. \nz 2021 r. poz. 1128, z późn. zm.2)). \n[2. Do kwoty doda tkowego rocznego świadczenia pieniężnego stosuje się \nodpowiednio art.  66 ust. 1 pkt 16, art. 81 ust.  8 pkt 2 i art. 85 ust. 9–11 ustawy \nz dnia 27  sierpnia 2004  r. o świadczeniach opieki zdrowotnej finansowanych \nze środków publicznych (Dz.  U. z 2021  r. poz.  1285, z późn. zm.3)).]  \n<2. Do kwoty dodatkowego rocznego świadczenia pieniężnego stosuje się \nodpowiednio art.  66 ust. 1 pkt 16, art.  81 ust. 8 pkt 2, art. 83 ust. 1 \ni art. 85 ust. 9–11 ustawy z  dnia 27  sierpnia 2004  r. o świadczeniach opieki \n                                                           \n1) Zmiany tekstu jednolitego wymienionej ustawy zostały ogłoszone w Dz. U. z 2021 r. poz. 2270 \noraz z 2022 r. poz. 1, 66, 1079, 1692, 1700, 1812, 1967 , 2127 i 2140 . \n2) Zmiany tekstu jednolitego wymienionej ustawy zostały ogłoszone w Dz. U. z 2021 r. poz. 1163, \n1243, 1551, 1574, 1834, 1981, 2071, 2105, 2133, 2232, 2269, 2270, 2328, 2376, 2427, 2430 i \n2490 oraz z 2022 r. poz. 1, 24, 64, 138, 501, 558, 583, 646, 655, 830, 872, 1079, 1265, 1301, \n1358, 1459, 1512, 1561, 1692, 1846, 1967, 2014, 2127 i 2180 . \n3) Zmiany tekstu jednolitego wymienionej ustawy zostały ogłoszone w Dz. U. z 2021 r. poz. 1292, \n1559, 1773, 1834, 1981, 2105, 2120, 2232, 2270, 2427 i 2469 oraz z 2022 r. poz. 64, 91, 526, \n583, 65 5, 807, 974, 1002, 1079, 1265, 1352 , 1700, 1855, 2140 i 2180.  Nowe brzmienie \nust. 2 w art. 12 \nwejdzie w życie z \ndn. 1.01.2024 r. \n(Dz. U. z 2023 r. \npoz. 1407). "

# Replace valid split points with a unique delimiter
modified_text = re.sub(r'(?<!Art\. \d+\.\s)(?<=\d\.\s)', '###SPLIT###', text)

# Split the text at the delimiter
split_text = modified_text.split('###SPLIT###')

# Displaying the split text
split_text


In [10]:
!pip install PyPDF2


14069.99s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 21.0 MB/s eta 0:00:00


In [12]:
import PyPDF2


def extract_text_from_pdf(pdf_path):
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(pdf_path)
    text_by_page = []

    # Extract text from each page
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
        text_by_page.append(text)

    return text_by_page


def identify_footer(text_by_page):
    footer_candidates = {}

    # Check the last few lines of each page to find repetitive text patterns
    for page_text in text_by_page:
        lines = page_text.split('\n')
        if len(lines) > 5:  # Ensure there are enough lines to check
            footer_line = lines[-1].strip()  # Get the last line
            if footer_line in footer_candidates:
                footer_candidates[footer_line] += 1
            else:
                footer_candidates[footer_line] = 1

    # Identify the most common footer line
    footer_line, count = max(footer_candidates.items(), key=lambda item: item[1])

    # Assume footer is the line that appears on most pages
    if count > 1:
        return footer_line
    return None


def remove_footer(text_by_page, footer_line):
    cleaned_text_by_page = []

    for page_text in text_by_page:
        cleaned_text = page_text.replace(footer_line, '').strip()
        cleaned_text_by_page.append(cleaned_text)

    return cleaned_text_by_page


def main(pdf_path):
    text_by_page = extract_text_from_pdf(pdf_path)
    footer_line = identify_footer(text_by_page)

    if footer_line:
        print(f"Identified footer: '{footer_line}'")
        cleaned_text_by_page = remove_footer(text_by_page, footer_line)
    else:
        print("No repetitive footer found.")
        cleaned_text_by_page = text_by_page

    # Print cleaned text for each page
    for page_num, text in enumerate(cleaned_text_by_page):
        print(f"Page {page_num + 1}:\n{text}\n")


if __name__ == "__main__":
    pdf_path = "D19910024Lj.pdf"
    main(pdf_path)


Identified footer: 'poz. 1429).'
Page 1:
©Kancelaria Sejmu     s. 1/75 
   
2023 -08-10 
  
Dz. U. 1991 Nr 7 poz. 24   
 
 
U S T AWA  
z dnia 20  grudnia 1990  r. 
o ubezpieczeniu społecznym rolników1) 
Rozdział 1 
Przepisy ogólne  
Art. 1. 1. Ubezpieczenie społeczne rolników, zwane dalej „ubezpieczeniem”, 
obejmuje, na zasadach określonych w  ustawie, rolników i  pracujących z  nimi 
domowników oraz pomocników rolnika, którzy:  
1) posiadaj ą obywatelstwo polskie lub  
2) są uprawnieni do wykonywania pracy na terytorium Rzeczypospolitej Polskiej 
na podstawie art.  87 ustawy z  dnia 20  kwietnia 2004  r. o promocji zatrudnienia 
i instytucjach rynku pracy (Dz.  U. z  2022  r. poz.  690, z późn. zm.2)) lub są 
zwolnieni na podstawie przep isów szczególnych z  obowiązku posiadania 
zezwolenia na pracę.  
1a. (uchylony)  
2. W ubezpieczeniu wyodrębnia się:  
1) ubezpieczenie wypadkowe, chorobowe i  macierzyńskie;  
2) ubezpieczenie emerytalno -rentowe.  
Art. 2. 1. Ubezpieczenie realizu